In [ ]:
# Washington Multiple AGN (WMAGN) Catalog DR1 - Double Peaked SDSS/Optical Sources
# Author: R. W. Pfeifle
# Original Date Created: 10 Sept. 2020
# Last Revision Date: 2 Oct. 2020

# New Form Creation Date: 13 January 2023
# Last Revision: 3 October 2023

# Purpose: Combine various catalogs of double-peaked optically selected dual AGN candidates
# 

In [ ]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [ ]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [ ]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2



In [ ]:
#tunique

#J013555.82+143529.7
#J101143.92+325943.5
#J111054.90+012936.0
#J123524.95+060810.7
#J131515.94+213403.6
#J150053.92+382349.6
#J155205.93+043317.5
#J225420.99-005134.1
#J230442.82-093345.3
#J231051.95-090011.9

In [ ]:
# Here we're loading in the double-peaked emission line galaxy catalog of Wang+2009
wang2009 = ((Table.read('Tables/Wang2009/table1.dat', readme = 'Tables/Wang2009/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---'])
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Since the format of the catalog will require names for both components, we'll add in a column 'Name2' which for \
# these and similar targets will be duplicates of the first 'Name column'. Same goes for z2, etc. 

wang2009['Name2'] = wang2009['Name']
wang2009['z2'] = wang2009['z']
wang2009['z1_type'] = "spec"
wang2009['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(wang2009,wang2009['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = wang2009['RA'], dec = wang2009['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
wang2009['RA1_deg'] = coordconvert.ra.degree
wang2009['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
wang2009['RA2'] = wang2009['RA']
wang2009['Dec2'] = wang2009['Dec']

wang2009['RA2_deg'] = wang2009['RA1_deg']
wang2009['Dec2_deg'] = wang2009['Dec1_deg']

# Adding details about the coordinates
wang2009['Equinox'] = "J2000"
wang2009['Coordinate_waveband'] = "Optical"
wang2009['Coordinate_Source'] = "SDSS"

wang2009['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
wang2009['Brightness1'] = -100
wang2009['Brightness_band1'] = -100
wang2009['Brightness_type1'] = -100

wang2009['Brightness2'] = -100
wang2009['Brightness_band2'] = -100
wang2009['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
wang2009['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#wang2009['Sep(kpc)'] = wang2009['Sep']*((cosmo.arcsec_per_kpc_proper(wang2009['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#wang2009['delta_z'] = wang2009['z']-wang2009['z2']
wang2009['dV'] = (2.99e+5)*((1+wang2009['z'])**2 - (1+wang2009['z2'])**2)/((1+wang2009['z'])**2+(1+wang2009['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
wang2009['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
wang2009['Confirmation Method'] = "-99"
wang2009['Paper(s)'] = "Wang+2009"
wang2009['BibCode(s)'] = "2009ApJ...705L..76W"
wang2009['DOI(s)'] = "https://doi.org/10.1088/0004-637X/705/1/L76"

wang2009['Notes'] = ''

# And dropping any columns that we don't need....
wang2009.drop(labels=['e_z','Del1','e_Del1','Del2','e_Del2','FOIII1','e_FOIII1','FOIII2','e_FOIII2','D'],\
              axis=1, inplace=True)


In [ ]:
#wang2009

In [ ]:
# Loading in the catalog from Liu+2010a now...
liu2010 = ((Table.read('Tables/XLiu2010a/table1.dat', readme = 'Tables/XLiu2010a/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])
# Note here that any cells containing '--' in Xin's table are going to be replaced by 0's

liu2010['Name'] = liu2010['SDSS']
liu2010['Name2'] = liu2010['SDSS']
liu2010['z2'] = liu2010['z']
liu2010['z1_type'] = "spec"
liu2010['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(liu2010,liu2010['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = liu2010['RA'], dec = liu2010['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
liu2010['RA1_deg'] = coordconvert.ra.degree
liu2010['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
liu2010['RA2'] = liu2010['RA']
liu2010['Dec2'] = liu2010['Dec']

liu2010['RA2_deg'] = liu2010['RA1_deg']
liu2010['Dec2_deg'] = liu2010['Dec1_deg']

# Adding details about the coordinates
liu2010['Equinox'] = "J2000"
liu2010['Coordinate_waveband'] = "Optical"
liu2010['Coordinate_Source'] = "SDSS"

liu2010['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
liu2010['Brightness1'] = -100
liu2010['Brightness_band1'] = -100
liu2010['Brightness_type1'] = -100

liu2010['Brightness2'] = -100
liu2010['Brightness_band2'] = -100
liu2010['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
liu2010['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#liu2010['Sep(kpc)'] = liu2010['Sep']*((cosmo.arcsec_per_kpc_proper(liu2010['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#liu2010['delta_z'] = liu2010['z']-liu2010['z2']
liu2010['dV'] = (2.99e+5)*((1+liu2010['z'])**2 - (1+liu2010['z2'])**2)/((1+liu2010['z'])**2+(1+liu2010['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
liu2010['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
liu2010['Confirmation Method'] = "-99"
liu2010['Paper(s)'] = "Liu+2010a"
liu2010['BibCode(s)'] = "2010ApJ...708..427L"
liu2010['DOI(s)'] = "https://doi.org/10.1088/0004-637X/708/1/427"

liu2010['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
liu2010.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
              axis=1, inplace=True)

#liu2010

In [ ]:
# Alright, so for whatever reason, there are 9 objects that do match across the Wang+2009 and Liu+2010 catalogs, \
# but astropy cannot match them together. Annoyingly, it's faster to just manually adjust the tables here after \
# the matching process, so that's what I'm doing below. 

# I'll be manually checking every matching process to ensure proper matches have been made and we are not \
# including duplicates

tunique, tmatches, idx1, idx2 = match_tables_fib(wang2009,liu2010,5)

# These are the 9 in the tunique table that will need adjustment. I guess so long as I just remove the Wang+ \
# entries, we'll be fine. 
objs_liu=['J013555.82+143529.7','J101143.92+325943.5','J111054.90+012936.0','J123524.95+060810.7',\
      'J131515.94+213403.6','J150053.92+382349.6','J155205.93+043317.5','J225420.99-005134.1',\
      'J230442.82-093345.3','J231051.95-090011.9']
objs_wang=['J013555+143529','J101143+325943','J111054+012936','J123524+060810',\
      'J131515+213403','J150053+382349','J155205+043317','J225420-005134',\
      'J230442-093345','J231051-090011']

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Table_flag']=='Table2':
        tmatches.at[index, 'Paper(s)'] = 'Wang+2009 ; ' + tmatches.at[index, 'Paper(s)']
        tmatches.at[index, 'BibCode(s)'] = '2009ApJ...705L..76W ; ' + tmatches.at[index, 'BibCode(s)']
        tmatches.at[index, 'DOI(s)'] = 'https://doi.org/10.1088/0004-637X/705/1/L76 ; ' + tmatches.at[index, 'DOI(s)']

# Now clipping out all Wang+2009 rows from the matches table
tmatches = tmatches[tmatches['Table_flag']!='Table1'].reset_index(drop=True)
# Adding the Wang+2009 information to the Liu+2010 rows in the unique table that are actually matches
for index, row in tunique.iterrows():
    if row['Name'] in objs_liu:
        tunique.at[index, 'Paper(s)'] = 'Wang+2009 ; ' + tunique.at[index, 'Paper(s)']
        tunique.at[index, 'BibCode(s)'] = '2009ApJ...705L..76W ; ' + tunique.at[index, 'BibCode(s)']
        tunique.at[index, 'DOI(s)'] = 'https://doi.org/10.1088/0004-637X/705/1/L76 ; ' + tunique.at[index, 'DOI(s)']
# Clipping out the Wang+2009 rows from the unique table that are actually matches 
tunique = tunique[~tunique['Name'].isin(objs_wang)].reset_index(drop=True)

# Concatenating everything together to generate a master table here
the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
the_whills.drop(labels=['index'], axis=1, inplace=True) #'level_0'

#the_whills


In [ ]:
#for i,j in zip(idx1,idx2):
#    print(i,j)
    

In [ ]:
## Alright, so for whatever reason, there are 9 objects that do match across the Wang+2009 and Liu+2010 catalogs, \
## but astropy cannot match them together. Annoyingly, it's faster to just manually adjust the tables here after \
## the matching process, so that's what I'm doing below. 
#
## I'll be manually checking every matching process to ensure proper matches have been made and we are not \
## including duplicates
#
#tunique, tmatches, idx1, idx2 = match_tables_fib(wang2009,liu2010,5)
#
## These are the 9 in the tunique table that will need adjustment. I guess so long as I just remove the Wang+ \
## entries, we'll be fine. 
#objs_liu=['J013555.82+143529.7','J101143.92+325943.5','J111054.90+012936.0','J123524.95+060810.7',\
#      'J131515.94+213403.6','J150053.92+382349.6','J155205.93+043317.5','J225420.99-005134.1',\
#      'J230442.82-093345.3','J231051.95-090011.9']
#objs_wang=['J013555+143529','J101143+325943','J111054+012936','J123524+060810',\
#      'J131515+213403','J150053+382349','J155205+043317','J225420-005134',\
#      'J230442-093345','J231051-090011']
#
## Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
#for index, row in tmatches.iterrows():
#    if row['Name']=='Liu+2010':
#        tmatches.at[index, 'Paper(s)'] = 'Wang+2009 ; ' + tmatches.at[index, 'Paper(s)']
#        tmatches.at[index, 'BibCode(s)'] = '2009ApJ...705L..76W ; ' + tmatches.at[index, 'BibCode(s)']
#        tmatches.at[index, 'DOI(s)'] = 'https://doi.org/10.1088/0004-637X/705/1/L76 ; ' + tmatches.at[index, 'DOI(s)']
#
## Now clipping out all Wang+2009 rows from the matches table
#tmatches = tmatches[tmatches['Paper(s)']!='Wang+2009'].reset_index(drop=True)
## Adding the Wang+2009 information to the Liu+2010 rows in the unique table that are actually matches
#for index, row in tunique.iterrows():
#    if row['Name'] in objs_liu:
#        tunique.at[index, 'Paper(s)'] = 'Wang+2009 ; ' + tunique.at[index, 'Paper(s)']
#        tunique.at[index, 'BibCode(s)'] = '2009ApJ...705L..76W ; ' + tunique.at[index, 'BibCode(s)']
#        tunique.at[index, 'DOI(s)'] = 'https://doi.org/10.1088/0004-637X/705/1/L76 ; ' + tunique.at[index, 'DOI(s)']
## Clipping out the Wang+2009 rows from the unique table that are actually matches 
#tunique = tunique[~tunique['Name'].isin(objs_wang)].reset_index(drop=True)
#
## Concatenating everything together to generate a master table here
#the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
#the_whills.drop(labels=['index'], axis=1, inplace=True) #'level_0'
#
##the_whills

In [ ]:
# Loading in the catalog from Smith+2010 now...
smith2010 = ((Table.read('Tables/Smith2010/table1.dat', readme = 'Tables/Smith2010/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])
# Note here that any cells containing '--' in Xin's table are going to be replaced by 0's

smith2010['Name'] = smith2010['SDSS']
smith2010['Name2'] = smith2010['SDSS']
smith2010['z2'] = smith2010['z']
smith2010['z1_type'] = "spec"
smith2010['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(smith2010,smith2010['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = smith2010['RA'], dec = smith2010['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
smith2010['RA1_deg'] = coordconvert.ra.degree
smith2010['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
smith2010['RA2'] = smith2010['RA']
smith2010['Dec2'] = smith2010['Dec']

smith2010['RA2_deg'] = smith2010['RA1_deg']
smith2010['Dec2_deg'] = smith2010['Dec1_deg']

# Adding details about the coordinates
smith2010['Equinox'] = "J2000"
smith2010['Coordinate_waveband'] = "Optical"
smith2010['Coordinate_Source'] = "SDSS"

smith2010['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
smith2010['Brightness1'] = -100
smith2010['Brightness_band1'] = -100
smith2010['Brightness_type1'] = -100

smith2010['Brightness2'] = -100
smith2010['Brightness_band2'] = -100
smith2010['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
smith2010['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#smith2010['Sep(kpc)'] = smith2010['Sep']*((cosmo.arcsec_per_kpc_proper(smith2010['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#smith2010['delta_z'] = smith2010['z']-smith2010['z2']
smith2010['dV'] = smith2010['Vel']
# smith+2010 included velocity measurements

# Adding information about the paper and the selection method
smith2010['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
smith2010['Confirmation Method'] = "-99"
smith2010['Paper(s)'] = "Smith+2010"
smith2010['BibCode(s)'] = "2010ApJ...716..866S"
smith2010['DOI(s)'] = "https://doi.org/10.1088/0004-637X/716/1/866"
smith2010['Notes'] = ''

#smith2010

In [ ]:
# Zhang+2016 examined a subset of the Smith+ objects, looking for evidence of statistically smaller \
# virial black hole masses in the binary candidates compared to single AGNs, but find larger masses instead.
# They disfavor these as binary candidates, but they do not list coordinates or designations.  They only lsit
# plate and fiber IDs. So I cannot match onto these unless I manually check for the IDs. It did not add much \ 
# so I am not adding it in for now.

#for index, row in thewhills.iterrows():
#    if row['Name'] in objs:
#        the_whills.at[index, 'Paper(s)'] += ' ; Zhang+2016'
#        the_whills.at[index, 'BibCode(s)'] += ' ; 2016MNRAS.457.3878Z' 
#        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stw210'
#        #the_whills.at[index, 'Notes']='AGNs confirmed via X-rays and reanalyses of optical spectroscopy.'
#        #the_whills.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'



In [ ]:
# Here we're matching the_whills against the Smith+2010 catalog

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,smith2010,5)

# Adding the DOI, author, and bibcode info to all of the Smith+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Table_flag']!='Table2':
        tmatches.at[index, 'Paper(s)'] += ' ; Smith+2010'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2010ApJ...716..866S' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/716/1/866'
#
##!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#
## We need to add in some commands that take the velocity column from the Smith+ rows and writes them into the \
## final row containing all relevant information
##!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#

# Now clipping out all Smith+2010 rows from the matches table
tmatches = tmatches[tmatches['Paper(s)']!='Smith+2010'].reset_index(drop=True)

# Concatenating everything together to generate a master table here
the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
the_whills.drop(labels=['index','Table_flag'], axis=1, inplace=True) #'level_0',



In [ ]:
print(the_whills['Name'][88])

In [ ]:
the_whills

In [ ]:
len(the_whills)

In [ ]:
# Add in Smith+2012 here...

# Wait, we didn't defer to Smith+2010 for names, so tomorrow just use the RA and Dec matching commands
ep_objs = ['J101241.20+215556.0','J113105.07+610405.1','J131018.47+250329.5','J144105.64+180507.9',\
           'J151518.29+551535.3','J153231.80+420342.7','J082857.99+074255.7','J123605.45-014119.1',\
           'J124928.36+353926.8','J133226.34+060627.3','J134415.75+331719.1','J144157.24+094859.1',\
           'J171544.02+600835.4','J081542.53+063522.9','J090615.92+121845.6','J091649.41+000031.5',\
           'J120343.22+283557.8','J121911.16+042905.9','J124813.82+362423.6',\
           'J130724.08+460400.9','J133455.24+612042.1','J145110.04+490813.5','J145408.36+240521.3',\
           'J153423.19+540809.0','J084049.46+272704.7',\
           'J120526.04+321314.6','J140500.14+073014.1',\
           'J140816.02+015528.3','J151842.95+244026.0','J210449.13-000919.1']

smith2012 = pd.DataFrame(data=ep_objs,columns=['SDSS'])
name_to_coords(smith2012,smith2012['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = smith2012['RA'], dec = smith2012['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
smith2012['RA1_deg'] = coordconvert.ra.degree
smith2012['Dec1_deg'] = coordconvert.dec.degree


tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,smith2012,5)

# Adding the DOI, author, and bibcode info to all of the Smith+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Table_flag']!='Table2':
        tmatches.at[index, 'Paper(s)'] += ' ; Smith+2012'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2012ApJ...752...63S' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/752/1/63'

# Now clipping out all Smith+2010 rows from the matches table
tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)

# Concatenating everything together to generate a master table here
the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
the_whills.drop(labels=['index','Table_flag'], axis=1, inplace=True) #'level_0',

#  Verified: There are 30 matching objects
the_whills

In [ ]:
print(the_whills['Name'].str[:7].to_list())


In [ ]:
import collections
print([item for item, count in collections.Counter(the_whills['Name'].str[:7].to_list()).items() if count > 1])


In [ ]:
#test = the_whills
#test = test[(test['Paper(s)']!='Liu+2010a') & (test['Paper(s)']!='Wang+2009') & (test['Paper(s)']!='Smith+2010') ]
#
#test

In [ ]:
# These are the duplicte entries that for some reason do not have matches. 10 matches; removing these, we will \
# recovder the correct number of 340 souble peaked objects across these three catalogs
#'J085416+502631' and 'J085416.76+502632.0' --> Wang+ and Liu+
#'J085841+104122' and 'J085841.76+104122.1' --> Wang+ and Liu+
#'J112659+294442' and 'J112659.54+294442.8' --> Wang+ and Liu+
#'J161006+210735' and 'J161006.42+210735.1' --> Wang+ and Liu+
#
#'J080218' and 'J080218.65+304622.7' --> Wang+ and Liu+
#'J082107' and 'J082107.89+502115.8' --> Wang+ and Liu+
#'J094427' and 'J094427.59+144717.1' --> Wang+ and Liu+
#'J110821+591851' and 'J110821.81+591852.0' --> Wang+ and Liu+
#'J111042' and 'J111042.36+030033.8' --> Wang+ and Liu+
#'J132547' and 'J132547.95+545019.5' --> Wang+ and Liu+ 


# Here we are manually updating the bib information and defering to Liu+ since these non-matches are an issue \
# between Liu+ and Wang+

# First, we're dropping the rows from Wang+


# Next, we're updating the doi information...


# These are the 9 in the tunique table that will need adjustment. I guess so long as I just remove the Wang+ \
# entries, we'll be fine. 

print(len(the_whills))

duplist = ['J080218.65+304622.7','J082107.89+502115.8','J094427.59+144717.1','J110821.81+591852.0',\
           'J111042.36+030033.8','J132547.95+545019.5','J085416.76+502632.0','J085841.76+104122.1',\
           'J112659.54+294442.8','J161006.42+210735.1']
droplist = ['J085416+502631','J085841+104122','J112659+294442','J161006+210735','J080218+304622',\
            'J082107+502115','J094427+144717','J110821+591851','J111042+030033','J132547+545019']
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name'] in duplist:
        the_whills.at[index, 'Paper(s)'] = 'Wang+2009 ; ' + the_whills.at[index, 'Paper(s)']
        the_whills.at[index, 'BibCode(s)'] = '2009ApJ...705L..76W ; ' + the_whills.at[index, 'BibCode(s)']
        the_whills.at[index, 'DOI(s)'] = 'https://doi.org/10.1088/0004-637X/705/1/L76 ; ' + the_whills.at[index, 'DOI(s)']

# Now clipping out all Wang+2009 rows from the matches table
the_whills = the_whills[~the_whills['Name'].isin(droplist)].reset_index(drop=True)

print(len(the_whills))




In [ ]:
the_whills['Name'].to_list()

In [ ]:
# Adding in Tingay & Wayth+2011

# As listed in Tingay+ table 1
#objs = ['J000249+004504','J095833-005118','J105653+331945','J110957+020138','J115249+190300',\
#        'J150452+321414','J151659+051751','J151709+335324','J152606+414014','J155619+094855',\
#        'J160024+264035']

# Made this list by manually checking my matched list because the Liu+ namings overwrite the Wang+ names

objs =  ['J000249.07+004504.8','J095833.20-005118.6','J105653+331945','J110957.14+020138.6','J115249.33+190300.3',\
         'J150452+321414','J151659.24+051751.5','J151709.21+335324.7','J152606+414014','J155619.30+094855.6',\
         'J160024+264035']
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Tingay+2011'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2011AJ....141..174T' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-6256/141/6/174'
        the_whills.at[index, 'Notes'] += ' Tingay+ find no evidence of double radio cores in VLBA imaging.'

# All of these have been manually adjusted and do match with the table.
# The weird issue with J000249 appears to have been solved


In [ ]:
#the_whills

In [ ]:
# Here we're adding in the information from Rosario's works... 

# First is J151709.21+335324.7, which Rosario+2010 notes as J151709.20+335324.7
# This object is in both the smith and liu catalogs

objs =  ['J151709.21+335324.7']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Rosario+2010'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2010ApJ...716..131R' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/716/1/131'
        the_whills.at[index, 'Notes'] += ' Rosario+ find that the radio jet and the emission-line regions are coaligned and the emission line spectrum is consistent with ionization by strong shocks. The profiles show a mirrior symmetry consistent with acceleration due to a bipolar outflow. There is also no evidence for a merger. The double-peak profiles are most likely due to jet-ISM interactions and not dual AGNs.'

# Now for the second object that they discussed in the appendix: J112939.77+605742.5, which they refer to as
# J112939.78+605742.6

objs =  ['J112939.77+605742.5']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Rosario+2010'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2010ApJ...716..131R' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/716/1/131'
        the_whills.at[index, 'Notes'] += ' Rosario+ find that a compact triple radio structure where the central source is cospatial with the galaxy nucleus. They identify this as the core of the jet. to jet-ISM interactions and not dual AGNs. The size of the radio source is smaller than in J1517; due to the compactness they cannot unambiguously associate the emission line gas and the jets but the double peaked emission lines in this system are again likely to be due to jet-ISM interactions.'


# Naming was manually adjusted to match that used in the OG double-peaked catalogs
# I have verified that the matching works
 

In [ ]:
#the_whills['Paper(s)'][280]

In [ ]:
# And now for Rosario+2011 which looked at near-IR AO Keck imaging for 12 double peaked AGNs

objs = ['J153231.80+420342.7','J091649.41+000031.5','J161027.41+130806.8','J081542.53+063522.9',\
        'J095207.62+255257.2','J130724.08+460400.9','J020011.52-093126.1','J140923.51-012430.5',\
        'J124859.72-025730.7','J154107.81+203608.8','J121911.16+042905.9','J072554.42+374436.9']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #print('True', row['Name'])
        the_whills.at[index, 'Paper(s)'] += ' ; Rosario+2011'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2011ApJ...739...44R' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/739/1/44'
        #the_whills.at[index, 'Notes'] += ' '

# Doubles here
the_whills.loc[the_whills['Name']=='J161027.41+130806.8', 'Sep'] = 2.35
the_whills.loc[the_whills['Name']=='J095207.62+255257.2', 'Sep'] = 1.00
the_whills.loc[the_whills['Name']=='J130724.08+460400.9', 'Sep'] = 2.37
the_whills.loc[the_whills['Name']=='J020011.52-093126.1', 'Sep'] = 1.17
the_whills.loc[the_whills['Name']=='J124859.72-025730.7', 'Sep'] = 0.53
the_whills.loc[the_whills['Name']=='J154107.81+203608.8', 'Sep'] = 2.00

the_whills.loc[the_whills['Name']=='J161027.41+130806.8', 'Notes'] += ' Double structure observed in Keck AO near-IR imaging.'
the_whills.loc[the_whills['Name']=='J095207.62+255257.2', 'Notes'] += ' Double structure observed in Keck AO near-IR imaging. Rosario+ consider it a merger.'
the_whills.loc[the_whills['Name']=='J130724.08+460400.9', 'Notes'] += ' Double structure observed in Keck AO near-IR imaging. Rosario+ consider it a merger.'
the_whills.loc[the_whills['Name']=='J020011.52-093126.1', 'Notes'] += ' Double structure observed in Keck AO near-IR imaging. Rosario+ consider it a merger. Outflow may be responsible for observed line profiles given the low level of ionization in both peaks (suggesting they both come from a common ionized region).'
the_whills.loc[the_whills['Name']=='J124859.72-025730.7', 'Notes'] += ' Double structure observed in Keck AO near-IR imaging. Rosario+ consider it a merger.'
the_whills.loc[the_whills['Name']=='J154107.81+203608.8', 'Notes'] += ' Double structure observed in Keck AO near-IR imaging.'

the_whills.loc[the_whills['Name']=='J153231.80+420342.7', 'Notes'] += ' Single structure observed in Keck AO near-IR imaging.'
the_whills.loc[the_whills['Name']=='J091649.41+000031.5', 'Notes'] += ' Single structure observed in Keck AO near-IR imaging.'
the_whills.loc[the_whills['Name']=='J081542.53+063522.9', 'Notes'] += ' Single structure observed in Keck AO near-IR imaging.'
the_whills.loc[the_whills['Name']=='J140923.51-012430.5', 'Notes'] += ' Single structure observed in Keck AO near-IR imaging.'
the_whills.loc[the_whills['Name']=='J121911.16+042905.9', 'Notes'] += ' Single structure observed in Keck AO near-IR imaging.'
the_whills.loc[the_whills['Name']=='J072554.42+374436.9', 'Notes'] += ' Single structure observed in Keck AO near-IR imaging.'


# Verified that the matching process works here


In [ ]:
# Loading in Shen+2011 information now...

shen2011 = pd.read_csv('Tables/Shen2011/Shen2011.csv', sep=',')

print(len(shen2011['Name'].to_list()))

# objs = ['J153231.80+420342.7','J091649.41+000031.5','J161027.41+130806.8','J081542.53+063522.9','J095207.62+255257.2','J130724.08+460400.9','J020011.52−093126.1','J140923.51−012430.5','J124859.72−025730.7','J154107.81+203608.8','J121911.16+042905.9','J072554.42+374436.9']
for index, row in the_whills.iterrows():
    if row['Name'] in shen2011['Name'].to_list():
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Shen+2011'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2011ApJ...735...48S' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/735/1/48'
        the_whills.at[index, 'Notes'] += ' '

# Iterate through and add notes for objects resolved and not resolved into two nuclei in near-IR imaging
# Also include angular/spectral separations

for index, row in the_whills.iterrows():
    # Check if the 'Name' exists in shen2011 dataframe
    if row['Name'] in shen2011['Name'].to_list():
        # Find the corresponding value from shen2011 dataframe
        if shen2011.loc[shen2011['Name'] == row['Name'], 'Category'].iloc[0]=='NLR kinematics':
            # Add to the notes in the_whills dataframe
            the_whills.at[index, 'Notes'] += ' Shen+2011 attribute the double-peaked lines to NLR kinematics. Only a single nucleus is resolved in near-IR imaging and the spatial offset observed between the two velocity components of the emission lines is larger than >0.6''. If the double-peaks were due to two NLRs from dual AGNs there should be two nuclei visible in the imaging.'
            the_whills.at[index, 'System Type'] = 'Likely Single AGN'
        elif shen2011.loc[shen2011['Name'] == row['Name'], 'Category'].iloc[0]=='Ambiguous':
            # Add to the notes in the_whills dataframe
            the_whills.at[index, 'Notes'] += ' Shen+2011 find only a single resolved nucleus in near-IR imaging but the velocity separation of the emission lines are spatially offset by <0.4'' which is below the resolution limit of the near-IR imaging. These could be single AGNs or closer dual AGNs. Angular separation is derived from the separation observed between the velocity separated lines in the slit spectrum. '
            the_whills.at[index, 'Sep'] = shen2011.loc[shen2011['Name'] == row['Name'], 'Spec_offset_as'].iloc[0]
        elif shen2011.loc[shen2011['Name'] == row['Name'], 'Category'].iloc[0]=='Binary AGN':
            # Add to the notes in the_whills dataframe
            the_whills.at[index, 'Notes'] += ' Shen+2011 identify two nuclei in near-IR imaging that coincide with the two velocity components of the narrow emission lines. Shen+ classify these as binary AGNs.'
            the_whills.at[index, 'Sep'] = shen2011.loc[shen2011['Name'] == row['Name'], 'PANIC_offset_as'].iloc[0]
        else:
            Print('Encountered a problem!')
            
# Also note in the 'Notes' that the separation is given by "the two velocity components of the narrow line emission,
# measured from the emission peaks of the two velocity components in the slit spectrum" as per Shen+2011


# But I have verified that the matching process works properly

#the_whills

In [ ]:
the_whills.to_csv('the_whills_beforege2012.csv', sep=',', index=False)


In [ ]:
ge2012 = ((Table.read('Tables/Ge2012/table3.dat', readme = 'Tables/Ge2012/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])

# There seems to be minimal matches across the pp-AGN table and the dual core tables. I'm going to skip matching \
# these tables together since it won't add much info at all.
######################
#ge2012t6 = ((Table.read('Ge2012/table6.dat', readme = 'Ge2012/ReadMe', format='ascii.cds')).to_pandas())
#ge2012t7 = ((Table.read('Ge2012/table7.dat', readme = 'Ge2012/ReadMe', format='ascii.cds')).to_pandas())
#ge = ge2012t5.set_index('SDSS')
#ge = ge.join(ge2012t6.set_index('SDSS'), rsuffix = 2)
#ge = ge.join(ge2012t7.set_index('SDSS'), rsuffix = 3)
######################

# Here we're choosing only the Type Is and Type IIs from Ge+2012
ge2012 = ge2012[(ge2012['T']==1) | (ge2012['T']==2)]
# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(ge2012,ge2012['SDSS'])

ge2012['Name'] = ge2012['SDSS']
ge2012['Name2'] = ge2012['SDSS']
ge2012['z1'] = ge2012['z']
ge2012['z2'] = ge2012['z']
ge2012['z1_type'] = "spec"
ge2012['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(ge2012,ge2012['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = ge2012['RA'], dec = ge2012['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
ge2012['RA1_deg'] = coordconvert.ra.degree
ge2012['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
ge2012['RA2'] = ge2012['RA']
ge2012['Dec2'] = ge2012['Dec']

ge2012['RA2_deg'] = ge2012['RA1_deg']
ge2012['Dec2_deg'] = ge2012['Dec1_deg']

# Adding details about the coordinates
ge2012['Equinox'] = "J2000"
ge2012['Coordinate_waveband'] = "Optical"
ge2012['Coordinate_Source'] = "SDSS"

ge2012['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
ge2012['Brightness1'] = ge2012['rmag']
ge2012['Brightness_band1'] = "SDSS r-band"
ge2012['Brightness_type1'] = "mag"

ge2012['Brightness2'] = ge2012['rmag']
ge2012['Brightness_band2'] = "SDSS r-band"
ge2012['Brightness_type2'] = "mag"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
ge2012['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#*******
#******************the cosmo arcsec to kpc command needs to be fixed! It is deprecated apparently!
#ge2012['Sep(kpc)'] = ge2012['Sep']*((cosmo.arcsec_per_kpc_proper(ge2012['z']))**(-1))
#************
#*******


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#ge2012['delta_z'] = ge2012['z']-ge2012['z2']
ge2012['dV'] = (2.99e+5)*((1+ge2012['z'])**2 - (1+ge2012['z2'])**2)/((1+ge2012['z'])**2+(1+ge2012['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
ge2012['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
ge2012['Confirmation Method'] = "-99"
ge2012['Paper(s)'] = "Ge+2012"
ge2012['BibCode(s)'] = "2012ApJS..201...31G"
ge2012['DOI(s)'] = "https://doi:10.1088/0067-0049/201/2/31"
ge2012['Notes'] = ''

# Drop the rows
ge2012.drop_duplicates(subset='Name', keep='last', inplace=True)

rows_to_drop = ge2012[(ge2012['Name'] == 'J153753.64-005720.7') | (ge2012['Name'] == 'J142438.01-010547.3')].index
ge2012.drop(rows_to_drop, inplace=True)
ge2012.reset_index(drop='True', inplace=True)


#ge2012


In [ ]:
names = ge2012['Name'].to_list()
matched_pairs = find_matches(names)

for pair in matched_pairs:
    print(f"Match found: {pair[0]} and {pair[1]}")
    

In [ ]:
# Here we're matching the_whills against the Ge+2012 catalog

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,ge2012,18)

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Paper(s)']!='Ge+2012':
        tmatches.at[index, 'Paper(s)'] += ' ; Ge+2012'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2012ApJS..201...31G' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi:10.1088/0067-0049/201/2/31'

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#
# We need to add in some commands that take the velocity column from the Ge+ rows and writes them into the \
# final row containing all relevant information
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#

# Now clipping out all Smith+2010 rows from the matches table
tmatches = tmatches[tmatches['Paper(s)']!='Ge+2012'].reset_index(drop=True)

# Concatenating everything together to generate a master table here
the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
the_whills.drop(labels=['index'], axis=1, inplace=True)

#the_whills

# These are the duplicate rows when adding/matching Ge+ to the_whills
# I'm not sure why, but these objects are listed twice in Ge+'s table

# Match found: J110053.07+053017.0 and J110053.07+053017.0

# Match found: J142438.01-010547.2 and J142438.01-010547.3

# Match found: J153753.63-005720.6 and J153753.64-005720.7

# duplicates have been removed
# this matching process is verified



In [ ]:
def find_matches(names):
    name_dict = {}
    matches = []

    for name in names:
        first_6_chars = name[:7]
        if first_6_chars in name_dict:
            matches.append((name_dict[first_6_chars], name))
        else:
            name_dict[first_6_chars] = name

    return matches

names = the_whills['Name'].to_list()
matched_pairs = find_matches(names)

for pair in matched_pairs:
    print(f"Match found: {pair[0]} and {pair[1]}")
    


In [ ]:
# Here now that we're loaded in the double peaked catalogs, we'll add in some individual targets

# First, Liu+2010b
liu2010objs = ['J110851.04+065901.4','J113126.08-020459.2','J114642.47+511029.6','J133226.34+060627.4']

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name'] in liu2010objs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Liu+2010b'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2010ApJ...715L..30L' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/2041-8205/715/1/L30'
        the_whills.at[index, 'Notes']='NIR nuclei and double OIII emitting regions are spatially coincident. Angular separations quoted are the separations between the OIII components.'
        the_whills.at[index, 'Confirmation Method'] = 'NIR Imaging / Optical Spectroscopy'

the_whills.loc[the_whills['Name']=='J110851.04+065901.4', 'Sep'] = 0.9
the_whills.loc[the_whills['Name']=='J113126.08-020459.2', 'Sep'] = 0.6
the_whills.loc[the_whills['Name']=='J114642.47+511029.6', 'Sep'] = 2.5
the_whills.loc[the_whills['Name']=='J133226.34+060627.4', 'Sep'] = 1.6

the_whills.loc[the_whills['Name']=='J110851.04+065901.4', 'System Type'] = 'Dual AGN'
the_whills.loc[the_whills['Name']=='J114642.47+511029.6', 'System Type'] = 'Dual AGN'

#the_whills


# Verified that this matching process works properly.
# Although I've just noticed that Smith+2012 shows up twice... odd. Maybe I matched twice by accident.


In [ ]:
#print(the_whills['Paper(s)'][223])

In [ ]:
# Now adding in formation from Nandi+2012 and Nandi+2017 for the misalgined/doube-peaked DDRG J1328

nandiobjs = ['J132848.46+275227.8']
for index, row in the_whills.iterrows():
    if row['Name'] in nandiobjs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Nandi+2012 ; Nandi+2017'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2012BASI...40..121N ; 2017MNRAS.467L..56N' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.48550/arXiv.1208.1941 ; https://doi.org/10.1093/mnrasl/slw256'
        the_whills.at[index, 'Notes'] += 'Nandi+12 flagged this as a misaligned DDRG and Nandi+2017 re-examined this in light of the double-peaked emission lines. They consider it a candidate binary. Inner structure is centered on optical host but misaligned with outer  structure by about 30 deg. Fot every double-peaked line they find different intensities for each component. The different component sshow similar BPT classifications/positions.  They argue this is evidence against  a rotating disc of jet-cloud interaction. The outer double shows steeper spectral indices than the inner double indicating two epochs of jet activity. '
        the_whills.loc[the_whills['Name']=='J132848.46+275227.8', 'System Type'] = 'Dual AGN / Binary AGN'


# Verified that this matching process works


In [ ]:
#the_whills

In [ ]:
# This is for Fu+2011

fu2011d = pd.read_csv('Tables/Fu2011a/fu2011_doubles.csv', sep=',')

# Converting the coordinates
coordconvert = SkyCoord(ra = fu2011d['RA'], dec = fu2011d['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
fu2011d['RA1_deg'] = coordconvert.ra.degree
fu2011d['Dec1_deg'] = coordconvert.dec.degree

#fu2011d


In [ ]:
# Here we will add a matching code that matches based on RA and DEC instead of by name because Fu+ did not \
# adopt the same naming convention as any previous work

# Here we're matching for the pairs in Fu+2011

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2011d,3)

#len(tmatches)

# This finds 16 matches, which is exactly what we'd expect (there are 16 doubles in the table from Fu+2011)

for i, j in zip(idx1, idx2):
    #print("i:", i, "j:", j)
    #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
    #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
    the_whills.at[i, 'Sep'] = fu2011d.at[j, 'Sep_as']
    the_whills.at[i, 'Paper(s)'] += ' ; Fu+2011'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2011ApJ...733..103F' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/733/2/103'
    the_whills.at[i, 'dV'] = fu2011d.at[j, 'dV']
    the_whills.at[i, 'Notes'] += ' Fu+2011 companions within 3 arcseconds,'

# verified that this matching process works properly



In [ ]:
# This is for Fu+2011

fu2011s = pd.read_csv('Tables/Fu2011a/fu2011_singles.csv', sep=',')

# Converting the coordinates
coordconvert = SkyCoord(ra = fu2011s['RA'], dec = fu2011s['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
fu2011s['RA1_deg'] = coordconvert.ra.degree
fu2011s['Dec1_deg'] = coordconvert.dec.degree


#fu2011s


In [ ]:
# Here we will add a matching code that matches based on RA and DEC instead of by name because Fu+ did not \
# adopt the same naming convention as any previous work

# Here we're matching for the singles in Fu+2011

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2011s,12)

# There are 34 objects in Fu+2011 singles table, so the tmatches table should end up with 68 objects total
# We end up only with 34 when matching with like 5''. When we match by 12'', we do recover the missing two objects

# We are good to proceed, as I have visually verified that these matches are correct

#len(tmatches)

for i, j in zip(idx1, idx2):
    #print("i:", i, "j:", j)
    #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
    #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
    #the_whills.at[i, 'Sep'] = fu2011s.at[j, 'NIRC2sep(as)']
    the_whills.at[i, 'Paper(s)'] += ' ; Fu+2011'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2011ApJ...733..103F' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/733/2/103'
    the_whills.at[i, 'dV'] = fu2011s.at[j, 'dV']
    the_whills.at[i, 'Notes'] += ' Fu+2011 found no companions within 3 arcseconds,'

# verified that this matching process works properly




In [ ]:
#the_whills


In [ ]:
# Here we're loading in the tables from Fu+2012:

fu2012t2 = pd.read_csv('Tables/Fu2012/Fu2012_t2.csv', sep=',')
# Table 3 includes objects that have a companion within 3'' of the SDSS fiber position
fu2012t3 = pd.read_csv('Tables/Fu2012/Fu2012_t3.csv', sep=',')
# Table 4 includes objecs that DO NOT have a companion within the 3'' fiber position, and therfore the double-\
# peaked lines in these objects cannot be due to kpc-duals because we're not observing two NLRs
fu2012t4 = pd.read_csv('Tables/Fu2012/Fu2012_t4.csv', sep=',')

# From caption for Table 2:
#Objects are grouped according to the origin of the double-peaked [O iii] lines. Column 1: J2000 designation. Type-1s are indicated by stars. Column
#2: redshift. Column 3: velocity splitting between [O iii] λ5007 components: ΔV/c = [(1 + zr )2/(1 + zb)2 − 1]/[(1 + zr )2/(1 + zb)2 + 1], where zr and zb are
#the redshifts of the redshifted and blueshifted [O iii] components, respectively. Columns 4 and 5: [O iii] λ5007 luminosity in log(L ) for the blueshifted (b)
#and redshifted (r) line, corrected for Galactic extinction. Columns 6 and 7: [Oiii] λ5007 FWHMs, corrected for the σ = 65 km s−1 instrumental broadening.
#Column 8: Kellermann et al. (1989) radio loudness, R = Fν,5 GHz/Fν,4400A. Columns 9 and 10: projected angular separation (arcsec) and physical separation
#(kpc) between the main components in a merging system. Column 11: secondary classification (Section 3). Column 12: classification from Shen et al. (2011a).

fu2012t2['Name'] = fu2012t2['SDSS Name']
fu2012t2['Name2'] = fu2012t2['SDSS Name']
#fu2012t2['z1'] = fu2012t2['z']
#fu2012t2['z2'] = fu2012t2['z']
#fu2012t2['z1_type'] = "spec"
#fu2012t2['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(fu2012t2,fu2012t2['SDSS Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = fu2012t2['RA'], dec = fu2012t2['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
fu2012t2['RA1_deg'] = coordconvert.ra.degree
fu2012t2['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
fu2012t2['RA2'] = fu2012t2['RA']
fu2012t2['Dec2'] = fu2012t2['Dec']

fu2012t2['RA2_deg'] = fu2012t2['RA1_deg']
fu2012t2['Dec2_deg'] = fu2012t2['Dec1_deg']

# Adding details about the coordinates
fu2012t2['Equinox'] = "J2000"
fu2012t2['Coordinate_waveband'] = "Optical"
fu2012t2['Coordinate_Source'] = "SDSS"

fu2012t2['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
fu2012t2['Brightness1'] = "-99"
fu2012t2['Brightness_band1'] = "-99"
fu2012t2['Brightness_type1'] = "-99"

fu2012t2['Brightness2'] = "-99"
fu2012t2['Brightness_band2'] = "-99"
fu2012t2['Brightness_type2'] = "-99"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
fu2012t2['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#*******
#******************the cosmo arcsec to kpc command needs to be fixed! It is deprecated apparently!
#fu2012t2['Sep(kpc)'] = fu2012t2['Sep']*((cosmo.arcsec_per_kpc_proper(fu2012t2['z']))**(-1))
#************
#*******


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#fu2012t2['delta_z'] = fu2012t2['z1']-fu2012t2['z2']
#fu2012t2['dV'] = (2.99e+5)*((1+fu2012t2['z1'])**2 - (1+fu2012t2['z2'])**2)/((1+fu2012t2['z1'])**2+(1+fu2012t2['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
fu2012t2['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
fu2012t2['Confirmation Method'] = "-99"
fu2012t2['Paper(s)'] = "Fu+2012"
fu2012t2['BibCode(s)'] = "2012ApJ...745...67F"
fu2012t2['DOI(s)'] = "https://doi.org/10.1088/0004-637X/745/1/67"

#fu2012t2

In [ ]:
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2012t2,5)

print(len(tmatches))


# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for i, j in zip(idx1, idx2):
    #print("i:", i, "j:", j)
    #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
    #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
    the_whills.at[i, 'Sep'] = fu2012t2.at[j, 'Deltatheta']
    the_whills.at[i, 'Paper(s)'] += ' ; Fu+2012'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2012ApJ...745...67F' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/745/1/67'
    the_whills.at[i, 'dV'] = fu2012t2.at[j, 'DeltaV']
    the_whills.at[i, 'Notes'] += ' Fu+2011 argue the origin of the double-peaked lines is ' + str(fu2012t2.at[j, 'Primary Class'])
    if pd.notna(fu2012t2.at[j, 'Secondary Class']):
        the_whills.at[i, 'Notes'] += 'Fu+2011 also include a secondary class: ' + str(fu2012t2.at[j, 'Secondary Class'])

# all match within 5''

# and I have now visually verified that we have the correct number of matches and that the \
# correct objects have been matched

#the_whills

# verified that this process works and adds in the notes as intended.
# ******
# However we still need to go back and add additonal notes/classifications based on Fu+2011 in-text descriptions
# ******



In [ ]:

fu2012t3['Name'] = fu2012t3['SDSSName']
fu2012t3['Name2'] = fu2012t3['SDSSName']
#fu2012t3['z1'] = fu2012t3['z']
#fu2012t3['z2'] = fu2012t3['z']
#fu2012t3['z1_type'] = "spec"
#fu2012t3['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(fu2012t3,fu2012t3['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = fu2012t3['RA'], dec = fu2012t3['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
fu2012t3['RA1_deg'] = coordconvert.ra.degree
fu2012t3['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
fu2012t3['RA2'] = fu2012t3['RA']
fu2012t3['Dec2'] = fu2012t3['Dec']

fu2012t3['RA2_deg'] = fu2012t3['RA1_deg']
fu2012t3['Dec2_deg'] = fu2012t3['Dec1_deg']

# Adding details about the coordinates
fu2012t3['Equinox'] = "J2000"
fu2012t3['Coordinate_waveband'] = "Optical"
fu2012t3['Coordinate_Source'] = "SDSS"

fu2012t3['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
fu2012t3['Brightness1'] = "-99"
fu2012t3['Brightness_band1'] = "-99"
fu2012t3['Brightness_type1'] = "-99"

fu2012t3['Brightness2'] = "-99"
fu2012t3['Brightness_band2'] = "-99"
fu2012t3['Brightness_type2'] = "-99"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
fu2012t3['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#*******
#******************the cosmo arcsec to kpc command needs to be fixed! It is deprecated apparently!
#fu2012t3['Sep(kpc)'] = fu2012t3['Sep']*((cosmo.arcsec_per_kpc_proper(fu2012t3['z']))**(-1))
#************
#*******


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#fu2012t3['delta_z'] = fu2012t3['z1']-fu2012t3['z2']
#fu2012t3['dV'] = (2.99e+5)*((1+fu2012t3['z1'])**2 - (1+fu2012t3['z2'])**2)/((1+fu2012t3['z1'])**2+(1+fu2012t3['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
fu2012t3['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
fu2012t3['Confirmation Method'] = "-99"
fu2012t3['Paper(s)'] = "Fu+2012"
fu2012t3['BibCode(s)'] = "2012ApJ...745...67F"
fu2012t3['DOI(s)'] = "https://doi.org/10.1088/0004-637X/745/1/67"

#fu2012t3


In [ ]:
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2012t3,5)

print(len(tmatches))


# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Fu+2012'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2012ApJ...745...67F' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/745/1/67'
        the_whills.at[index, 'Notes'] += ' Companion(s) within 3 arcseconds.'


# all match within 5''

# and I have now visually verified that we have the correct number of matches and that the \
# correct objects have been matched

#the_whills

In [ ]:
#tmatches = tmatches.sort_values(by=['z'])
#tmatches

In [ ]:

fu2012t4['Name'] = fu2012t4['SDSS']
fu2012t4['Name2'] = fu2012t4['SDSS']
#fu2012t4['z1'] = fu2012t4['z']
#fu2012t4['z2'] = fu2012t4['z']
#fu2012t4['z1_type'] = "spec"
#fu2012t4['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(fu2012t4,fu2012t4['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = fu2012t4['RA'], dec = fu2012t4['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
fu2012t4['RA1_deg'] = coordconvert.ra.degree
fu2012t4['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
fu2012t4['RA2'] = fu2012t4['RA']
fu2012t4['Dec2'] = fu2012t4['Dec']

fu2012t4['RA2_deg'] = fu2012t4['RA1_deg']
fu2012t4['Dec2_deg'] = fu2012t4['Dec1_deg']

# Adding details about the coordinates
fu2012t4['Equinox'] = "J2000"
fu2012t4['Coordinate_waveband'] = "Optical"
fu2012t4['Coordinate_Source'] = "SDSS"

fu2012t4['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
fu2012t4['Brightness1'] = "-99"
fu2012t4['Brightness_band1'] = "-99"
fu2012t4['Brightness_type1'] = "-99"

fu2012t4['Brightness2'] = "-99"
fu2012t4['Brightness_band2'] = "-99"
fu2012t4['Brightness_type2'] = "-99"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
fu2012t4['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#*******
#******************the cosmo arcsec to kpc command needs to be fixed! It is deprecated apparently!
#fu2012t4['Sep(kpc)'] = fu2012t4['Sep']*((cosmo.arcsec_per_kpc_proper(fu2012t4['z']))**(-1))
#************
#*******


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#fu2012t4['delta_z'] = fu2012t4['z1']-fu2012t4['z2']
#fu2012t4['dV'] = (2.99e+5)*((1+fu2012t4['z1'])**2 - (1+fu2012t4['z2'])**2)/((1+fu2012t4['z1'])**2+(1+fu2012t4['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
fu2012t4['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
fu2012t4['Confirmation Method'] = "-99"
fu2012t4['Paper(s)'] = "Fu+2012"
fu2012t4['BibCode(s)'] = "2012ApJ...745...67F"
fu2012t4['DOI(s)'] = "https://doi.org/10.1088/0004-637X/745/1/67"

#fu2012t4


In [ ]:
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2012t4,14)

print(len(tmatches))


# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Fu+2012'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2012ApJ...745...67F' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/745/1/67'
        the_whills.at[index, 'Notes'] += ' No companion within 3 arcseconds.'


# we have to go out to 14'' in match tolerance \
# to get the last object (72 match within 10'')

# and I have now visually verified that we have the correct number of matches and that the \
# correct objects have been matched

#the_whills

In [ ]:
# def find_matches(names):
#     name_dict = {}
#     matches = []

#     for name in names:
#         first_6_chars = name[:7]
#         if first_6_chars in name_dict:
#             matches.append((name_dict[first_6_chars], name))
#         else:
#             name_dict[first_6_chars] = name

#     return matches

# names = the_whills['Name'].to_list()
# matched_pairs = find_matches(names)

# for pair in matched_pairs:
#     print(f"Match found: {pair[0]} and {pair[1]}")
    
# tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,fu2012t4,14)

# print(len(tmatches))


# # Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
# for index, row in the_whills.iterrows():
#     if index in idx1:
#         the_whills.at[index, 'Paper(s)'] += ' ; Fu+2012'
#         the_whills.at[index, 'BibCode(s)'] += ' ; 2012ApJ...745...67F' 
#         the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/745/1/67'
#         the_whills.at[index, 'Notes'] += ' No companion within 3 arcseconds.'


# # we have to go out to 14'' in match tolerance \
# # to get the last object (72 match within 10'')

# # and I have now visually verified that we have the correct number of matches and that the \
# # correct objects have been matched

# #the_whills
    

In [ ]:
# Here we're loading in the double-peaked emission line galaxy catalog of Comerford2012
comerford2012t1 = ((Table.read('Tables/Comerford2012/table1.dat', readme = 'Tables/Comerford2012/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---', 'T/I','Date1','---_1','Date2','PA1','PA2','Exp1','---_2','Exp2'])
comerford2012t2 = ((Table.read('Tables/Comerford2012/table2.dat', readme = 'Tables/Comerford2012/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['n_SName','T','Em','PA','e_PA','dPA','ell','e_dXang','dXkpc','e_dXkpc','e_dV'])

comerford2012 = (pd.concat([comerford2012t1,comerford2012t2], axis=1)).drop(columns=['SName'])


#References. (1) Liu et al. 2010b; (2) Wang et al. 2009; (3) Fu et al. 2012; (4) Shen et al. 2011; (5) Tingay & Wayth 2011; (6) Smith et al. 2010;
#(7) Rosario et al. 2011; (8) Fu et al. 2011a; (9) McGurk et al. 2011; (10) Rosario et al. 2010; (11) Comerford et al. 2011.

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(comerford2012,comerford2012['SDSS'])

comerford2012['Name'] = comerford2012['SDSS']
# Converting the coordinates
coordconvert = SkyCoord(ra = comerford2012['RA'], dec = comerford2012['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2012['RA1_deg'] = coordconvert.ra.degree
comerford2012['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
comerford2012['RA2'] = comerford2012['RA']
comerford2012['Dec2'] = comerford2012['Dec']

comerford2012['RA2_deg'] = comerford2012['RA1_deg']
comerford2012['Dec2_deg'] = comerford2012['Dec1_deg']


#comerford2012

In [ ]:
# Here we're matching comerford+2012

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,comerford2012,15)

print(len(tmatches))

for i, j in zip(idx1, idx2):
    #print("i:", i, "j:", j)
    #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
    #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
    the_whills.at[i, 'Sep'] = comerford2012.at[j, 'dXang']
    the_whills.at[i, 'Paper(s)'] += ' ; Comerford+2012'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2012ApJ...753...42C' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/753/1/42'
    the_whills.at[i, 'dV'] = comerford2012.at[j, 'dV']
    the_whills.at[i, 'Notes'] += ' Comerford+ examined long slit spectra of this target. Two spatially distinct emission peaks observed.'

# comerford+ consider 14 objects as being promising dual AGN candidates:
dualcand = ['J084049.47+272704.8','J095207.62+255257.2','J160524.59+152233.5',\
            'J163316.03+262716.3','J210449.13-000919.1','J225510.12-081234.4',\
            'J230442.82-093345.3','J011659.59-102539.1','J080418.23+305157.2',\
            'J093024.84+343057.3','J102325.57+324348.4','J144804.17+182537.9',\
            'J162939.58+240856.0','J225420.99-005134.1']
dualcand = pd.DataFrame(dualcand, columns=['SDSS'])

name_to_coords(dualcand,dualcand['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = dualcand['RA'], dec = dualcand['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
dualcand['RA1_deg'] = coordconvert.ra.degree
dualcand['Dec1_deg'] = coordconvert.dec.degree

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,dualcand,15)

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Notes'] += ' Comerford+2012 consider this a strong dual AGN candidate. Double optical emission line peaks observed oriented along the plane of the galaxy.'

# we have to use an exceptionally large search size (15''), but i visually verified that the matching is working

#tmatches = tmatches.sort_values(by=['Name'])
#tmatches


In [ ]:
dualcand = ['J095207.62+255257.2','J115523.74+150756.9','J123915.40+531414.6']
dualcand = pd.DataFrame(dualcand, columns=['SDSS'])

name_to_coords(dualcand,dualcand['SDSS'])


# Converting the coordinates
coordconvert = SkyCoord(ra = dualcand['RA'], dec = dualcand['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
dualcand['RA1_deg'] = coordconvert.ra.degree
dualcand['Dec1_deg'] = coordconvert.dec.degree

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,dualcand,15)

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Notes'] += ' Comerford+2012 consider this a very strong dual AGN candidate. The spatial separations and positional angles of the double emission components likely coincide with the observed double stellar nuclei.'




In [ ]:
the_whills.to_csv('the_whills_uptoComerford2013.csv', sep=',')


In [ ]:
# Here we're loading in the double-peaked emission line galaxy catalog of Comerford2013
comerford2013 = ((Table.read('Tables/Comerford2013/table1.dat', readme = 'Tables/Comerford2013/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---','HbVoff','e_HbVoff','O3Voff','e_O3Voff','HaVoff','e_HaVoff','O3/Hb','e_O3/Hb','N2/Ha','e_N2/Ha','S2/Ha','e_S2/Ha','O1/Ha','e_O1/Ha','Class','n_Class'])
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Comerford2013 looked at 173 Type 2 AGNs in Deep2 and found only two double-peaked dual AGN candidates \
# and found five offset AGNs. We are retaining only the double peaked duals.s

comerford2013 = comerford2013[(comerford2013['NDWFS'] == 'J143208.27+353255.5') | (comerford2013['NDWFS'] == 'J143359.71+351020.5')]

comerford2013['Coordinates'] = comerford2013['NDWFS'].str.slice(start=1)
comerford2013['RA_test'] = comerford2013['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
comerford2013['Dec_test'] = comerford2013['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
comerford2013['RA'] = comerford2013['RA_test'].str.slice(start=0, stop=2)+":"+comerford2013['RA_test'].str.slice(start=2, stop=4)+":"+comerford2013['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
comerford2013['Dec'] = comerford2013['Dec_test'].str.slice(start=0, stop=3)+":"+comerford2013['Dec_test'].str.slice(start=3, stop=5)+":"+comerford2013['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
comerford2013.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)


comerford2013['Name'] = comerford2013['NDWFS']
comerford2013['Name2'] = comerford2013['NDWFS']
comerford2013['z1'] = comerford2013['z']
comerford2013['z2'] = comerford2013['z']
comerford2013['z1_type'] = "spec"
comerford2013['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(comerford2013,comerford2013['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = comerford2013['RA'], dec = comerford2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2013['RA1_deg'] = coordconvert.ra.degree
comerford2013['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
comerford2013['RA2'] = comerford2013['RA']
comerford2013['Dec2'] = comerford2013['Dec']

comerford2013['RA2_deg'] = comerford2013['RA1_deg']
comerford2013['Dec2_deg'] = comerford2013['Dec1_deg']

# Adding details about the coordinates
comerford2013['Equinox'] = "J2000"
comerford2013['Coordinate_waveband'] = "Optical"
comerford2013['Coordinate_Source'] = "AGES"

comerford2013['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
comerford2013['Brightness1'] = "-99"
comerford2013['Brightness_band1'] = "-99"
comerford2013['Brightness_type1'] = "-99"

comerford2013['Brightness2'] = "-99"
comerford2013['Brightness_band2'] = "-99"
comerford2013['Brightness_type2'] = "-99"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
comerford2013['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#*******
#******************the cosmo arcsec to kpc command needs to be fixed! It is deprecated apparently!
#comerford2013['Sep(kpc)'] = comerford2013['Sep']*((cosmo.arcsec_per_kpc_proper(comerford2013['z']))**(-1))
#************
#*******


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
comerford2013['delta_z'] = comerford2013['z1']-comerford2013['z2']
comerford2013['dV'] = (2.99e+5)*((1+comerford2013['z1'])**2 - (1+comerford2013['z2'])**2)/((1+comerford2013['z1'])**2+(1+comerford2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
comerford2013['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
comerford2013['Confirmation Method'] = "-99"
comerford2013['Paper(s)'] = "Comerford+2013"
comerford2013['BibCode(s)'] = "2013ApJ...777...64C"
comerford2013['DOI(s)'] = "https://doi:10.1088/0004-637X/777/1/64"

comerford2013 = comerford2013.iloc[::2].reset_index(drop=True)
# Note: for some reason, if we don't drop the index here, the matching process ahead will not work and one of the matching objects between Ge+ and Comerford+ will be considered both unique and a match.
#comerford2013

In [ ]:
#comerford2013

In [ ]:
# Here we're matching the_whills against the Comerford2013 catalog

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,comerford2013,5)

#tmatches
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Paper(s)']!='Comerford+2013':
        tmatches.at[index, 'Paper(s)'] += ' ; Comerford+2013'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2013ApJ...777...64C' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi:10.1088/0004-637X/777/1/64'
        tmatches.at[index, 'Notes'] += ' Ge+ selected this as a double-peaked source in SDSS. Comerford+ selected this as a double-peaked dual candidate in AGES.'
        
# Now clipping out all Comerford+2013 rows from the matches table
tmatches = tmatches[tmatches['Paper(s)']!='Comerford+2013'].reset_index(drop=True)

# Concatenating everything together to generate a master table here
the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
the_whills.drop(labels=['level_0','index'], axis=1, inplace=True)
#tunique
#the_whills

# Verified that this matchinkg process works properly, and I debugged a previous issue related to the indexing/matching for the Comerford+ table.


In [ ]:
#the_whills


In [ ]:
# Here we're adding in the information from An+2013 which examines J151656.59+183021.5, a double-peaked AGN \
# identified in Smith+2010

an2013obs = ['J151656.59+183021.5']
for index, row in the_whills.iterrows():
    if row['Name'] in an2013obs:
        print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; An+2013'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2013MNRAS.433.1161A' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stt801'
        the_whills.at[index, 'Notes'] += ' An+ refers to this as 3C 316. An+ presents eMERLIN VLA and EVN imaging of 3C 316 and reveals three major components (eastern and central and western components) that are further resolved into discrete jet components in an S-shaped line in the EVN image. None of these could be unambiguously identified as an AGN core. These resemble the knots of jets. They argue the radio structure is consistent with a single radio emitting AGN but they cannot rule out a kiloparsec radio-quiet companion.'

# Verified that this matching process works


In [ ]:
#the_whills

In [ ]:
# Here we're loading in the double-peaked emission line galaxy catalog of Barrows2013
#barrows2013t1 = ((Table.read('Barrows2013/table1.dat', readme = 'Barrows2013/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---','HbVoff','e_HbVoff','O3Voff','e_O3Voff','HaVoff','e_HaVoff','O3/Hb','e_O3/Hb','N2/Ha','e_N2/Ha','S2/Ha','e_S2/Ha','O1/Ha','e_O1/Ha','Class','n_Class'])
#barrows2013t2 = ((Table.read('Barrows2013/table2.dat', readme = 'Barrows2013/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---','HbVoff','e_HbVoff','O3Voff','e_O3Voff','HaVoff','e_HaVoff','O3/Hb','e_O3/Hb','N2/Ha','e_N2/Ha','S2/Ha','e_S2/Ha','O1/Ha','e_O1/Ha','Class','n_Class'])
barrows2013 = ((Table.read('Tables/Barrows2013/table3.dat', readme = 'Tables/Barrows2013/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['e_zsdss','zmgii','e_zmgii','FWHMfeii','FWHMmgii','e_FWHMmgii','fedd'])
# table 1 and 2 specifically look at Ne V and Ne III double peaked sources (with some overlap)
# table 3 includes the full sample

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(barrows2013,barrows2013['SDSS'])

barrows2013['Name'] = barrows2013['SDSS']
barrows2013['Name2'] = barrows2013['SDSS']
barrows2013['z1'] = barrows2013['zsdss']
barrows2013['z2'] = barrows2013['zsdss']
barrows2013['z1_type'] = "spec"
barrows2013['z2_type'] = "spec"

# Converting the coordinates
coordconvert = SkyCoord(ra = barrows2013['RA'], dec = barrows2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
barrows2013['RA1_deg'] = coordconvert.ra.degree
barrows2013['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
barrows2013['RA2'] = barrows2013['RA']
barrows2013['Dec2'] = barrows2013['Dec']

barrows2013['RA2_deg'] = barrows2013['RA1_deg']
barrows2013['Dec2_deg'] = barrows2013['Dec1_deg']

# Adding details about the coordinates
barrows2013['Equinox'] = "J2000"
barrows2013['Coordinate_waveband'] = "Optical"
barrows2013['Coordinate_Source'] = "AGES"

barrows2013['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
barrows2013['Brightness1'] = "-99"
barrows2013['Brightness_band1'] = "-99"
barrows2013['Brightness_type1'] = "-99"

barrows2013['Brightness2'] = "-99"
barrows2013['Brightness_band2'] = "-99"
barrows2013['Brightness_type2'] = "-99"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
barrows2013['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#*******
#******************the cosmo arcsec to kpc command needs to be fixed! It is deprecated apparently!
#barrows2013['Sep(kpc)'] = barrows2013['Sep']*((cosmo.arcsec_per_kpc_proper(barrows2013['z']))**(-1))
#************
#*******

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#barrows2013['delta_z'] = barrows2013['z1']-barrows2013['z2']
barrows2013['dV'] = (2.99e+5)*((1+barrows2013['z1'])**2 - (1+barrows2013['z2'])**2)/((1+barrows2013['z1'])**2+(1+barrows2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
barrows2013['Selection Method'] = "Double-Peaked Spectroscopic Emission Lines" #DPSELs
barrows2013['Confirmation Method'] = "-99"
barrows2013['Paper(s)'] = "Barrows+2013"
barrows2013['BibCode(s)'] = "2013ApJ...769...95B"
barrows2013['DOI(s)'] = "https://doi:10.1088/0004-637X/769/2/95"
barrows2013['Notes'] = " "

#barrows2013

# Here we're matching the_whills against the Barrows+2013 catalog
#tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,barrows2013,5)
#tmatches
# There are no matches between Barrows+2013 and the_whills

# Run next cell to concatenate Barrows+ with the main table

In [ ]:
the_whills = pd.concat([the_whills,barrows2013])

# Verified that this matching/concatenation process is working


In [ ]:
# Here I'm going to add in the information from Liu+2013, Shangguan+2016, and \
# Barrows+2016, and +2018, and Bondi+2016

# Here now that we're loaded in the double peaked catalogs, we'll add in some individual targets

# First, Liu+2010b
liu2010objs = ['J110851.04+065901.4','J113126.08-020459.2','J114642.47+511029.6','J133226.34+060627.4']

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name'] in liu2010objs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Liu+2013 ; Shangguan+2016'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2013ApJ...762..110L ; 2016ApJ...823...50S' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/762/2/110 ; https://doi.org/10.3847/0004-637X/823/1/50'
        #the_whills.at[index, 'System Type']='Dual AGN'
        #the_whills.at[index, 'Notes']='NIR nuclei and double OIII emitting regions are spatially coincident. Angular separations quoted are the separations between the OIII components.'
        #the_whills.at[index, 'Confirmation Method'] = 'NIR Imaging / Optical Spectroscopy'
        
the_whills.loc[the_whills['Name']=='J110851.04+065901.4', 'System Type'] = 'Dual AGN'
the_whills.loc[the_whills['Name']=='J114642.47+511029.6', 'System Type'] = 'Dual AGN'


# Adding in additional info from Bondi+2016 here...
bondi2016 = ['J110851.04+065901.4','J113126.08-020459.2']

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name'] in bondi2016:
        #print('True2')
        the_whills.at[index, 'Paper(s)'] += ' ; Bondi+2016'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016A&A...588A.102B' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1051/0004-6361/201528021'
        the_whills.at[index, 'Notes'] += ' '

the_whills.loc[the_whills['Name']=='J110851.04+065901.4', 'Notes'] += " One single compact source is reported by Bondi+ in EVN observations."
the_whills.loc[the_whills['Name']=='J113126.08-020459.2', 'Notes'] += " Bondi+ report that the VLBI observation show no compact cores but the VLA imaging reveals a possible core."
        
# Now for Barrows+2016 # Borrows+ does not add much here beyond the literature besides also selecting this as an offset AGN candidate
barrowsobjs = ['J110851.04+065901.4'] # Barrows+2016 has is listed as J110851.04+065901.5
for index, row in the_whills.iterrows():
    if row['Name'] in barrowsobjs:
        #print('True3')
        the_whills.at[index, 'Paper(s)'] += ' ; Barrows+2016'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...829...37B' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/829/1/37'

# Now for Barrows+2016,2018
barrowsobjs = ['J123420.14+475155.9'] # Barrows lists it as SDSSJ123420.14+471555.86
for index, row in the_whills.iterrows():
    if row['Name'] in barrowsobjs:
        #print('True4')
        the_whills.at[index, 'Paper(s)'] += ' ; Barrows+2016 ; Barrows+2018'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...829...37B ; 2018ApJ...869..154B' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/829/1/37 ; https://doi.org/10.3847/1538-4357/aaedb6'
        the_whills.at[index, 'Notes'] += ' Barrows+2018 notes that the one stellar core is the bluest of stellar cores within their sample while and the other is reddened.'

#Also in Barrows 2016 (but not 2017 and 2018): J110851.04+065901.4
#Nevin+ does not overlap with these follow-up works but does overlap with the original double peak catalogs
#Liu+2018 will have its own cell because it examines 18 objects 
#Barrows  2016 and 2018 overlap for the object not  included in Liu+ --> this will have a separate entry in the manual table
#Barrows+2016,+2018 J1234 overlaps with Ge+2012 --> this we'll have a cell here for adding bib info

#******* THIS IS A NOTE TO COME BACK TO SO THAT BARROWS IS FULLY PROCESSED*****
#Barrows+2016 also overlaps with Liu+2011 catalog

#the_whills.loc[the_whills['Name']=='J110851.04+065901.4', 'Sep'] = 0.9
#the_whills.loc[the_whills['Name']=='J113126.08-020459.2', 'Sep'] = 0.6
#the_whills.loc[the_whills['Name']=='J114642.47+511029.6', 'Sep'] = 2.5
#the_whills.loc[the_whills['Name']=='J133226.34+060627.4', 'Sep'] = 1.6

# Barrows+2016 has a typo in the name of J123420.14 in their manuscript


# Verified that the matching process here works


In [ ]:
#Here we're adding in the results from Comerford+2015
# We adjusted the naming from comerford+ for J014209.01−005050.0
objs = ['J014209-005049','J075223.35+273643.1','J084135.09+010156.2','J085416.76+502632.0',\
        'J095207.62+255257.2','J100654.20+464717.2','J112659.54+294442.8','J123915.40+531414.6',\
        'J132231.86+263159.1','J135646.11+102609.1','J144804.17+182537.9','J160436.21+500958.1']
#print(len(objs))
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #print('True', row['Name'])
        the_whills.at[index, 'Paper(s)'] += ' ; Comerford+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015ApJ...806..219C' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/806/2/219'
        #the_whills.at[index, 'Notes']='AGNs confirmed via X-rays and reanalyses of optical spectroscopy.'
        #the_whills.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'

#the_whills.at[index, 'System Type']='Dual AGN'

objs = ['J112659.54+294442.8']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #print('True1', row['Name'])
        the_whills.at[index, 'System Type'] = 'Dual AGN'
        the_whills.at[index, 'Notes'] += 'Comerford+ claim a dual AGN in this system. It is an extremely minor merger (460:1) where the secondary is detected at 2.3sigma.'
        #the_whills.at[index, 'Notes'] += 'AGNs confirmed via X-rays and reanalyses of optical spectroscopy.'
        the_whills.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'

objs = ['J084135.09+010156.2','J095207.62+255257.2','J123915.40+531414.6','J132231.86+263159.1','J135646.11+102609.1']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #print('True2', row['Name'])
        the_whills.at[index, 'System Type']='Dual AGN Candidate/Offset AGN Candidate'
        the_whills.at[index, 'Notes'] += ' Comerford+ classify these as dual AGN candidates/Offset AGN candidates based on Chandra and HST imaging. These systems have two [OIII] emission components with the same spatial separation and orientation as the two stellar bulges (within 3sigma) but two coincident X-ray sources are not detected above 2sigma.'

objs = ['J014209-005049','J075223.35+273643.1','J085416.76+502632.0','J100654.20+464717.2','J144804.17+182537.9','J160436.21+500958.1']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #print('True3', row['Name'])
        the_whills.at[index, 'System Type']='Single AGN'
        the_whills.at[index, 'Notes'] += ' Comerford+2015 conclude these are likely single AGNs.'

objs = ['J095207.62+255257.2']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #print('True4', row['Name'])
        the_whills.at[index, 'System Type']='Dual AGN'
        the_whills.at[index, 'Notes'] += ' McGurk+2011 classified this as a Type 1 + type II pair based on extracted spectra.'
        # --> we may not need to actually add this note. We may have to just drop the target from the tables \
        # here and add it back in via an individual object listing

# Did not originally match. The former is because of the naming convention, the latter simply wasn't in the table
#J014209.01−005050.0
#J084135.09+010156.2

#
# We need to figure out why J084135.09+010156.2 does not show up
# I looked manually through the Wang+ Liu+ and Smith+ tables and I do not see J0841+0101 anywhere...
# I think it's not in those tables because it isn't actually a double-peaked source? I;m not \
# entirely sure why it was in Comerford+'s paper, but I think we're going to remove it from here and load \
# it in manually in the individual targets table


# Verified that the matching process works


In [ ]:
# Here we're addiing in the Gabanyi+ results that focus on four objects drawn from Comerford+2012

# SDSS J102325.57+324348.4 J1023+3243
# SDSS J115523.74+150756.9 J1155+1507
# SDSS J210449.13–000919.1 J2104–0009
# SDSS J23044.82–093345.3 J2304–0933


gabobjs = ['J210449.13-000919.1','J230442.82-093345.3'] # there was a typo in Gabanyi's writing of J230442
for index, row in the_whills.iterrows():
    if row['Name'] in gabobjs:
        #print('True1')
        the_whills.at[index, 'Paper(s)'] += ' ; Gabanyi+2016'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...826..106G' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/826/2/106'
        the_whills.at[index, 'Notes'] += ' Gabanyi+ find single radio sources in this object and that the position angles of the radio structure and double optical emission components are consistent and support a jet-driven scenario for the double peaked emission lines (though the radio jets influencing the lines are on much larger scales than mapped by Gabanyi+).'


gabobjs = ['J115523.74+150756.9'] # 
for index, row in the_whills.iterrows():
    if row['Name'] in gabobjs:
        #print('True2')
        the_whills.at[index, 'Paper(s)'] += ' ; Gabanyi+2016 '
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...826..106G' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/826/2/106'
        the_whills.at[index, 'Notes'] += ' Gabanyi+ find the position angles of the radio emission and optical emission are nearly perpendicular and are therefore unrelated. Only one source is detected in the radio imaging.'

gabobjs = ['J102325+324348'] # Gabanyi+ uses J102325.57+324348.4 for naming. I'm going to rename the listing in my table
for index, row in the_whills.iterrows():
    if row['Name'] in gabobjs:
        #print('True3')
        the_whills.at[index, 'Paper(s)'] += ' ; Gabanyi+2016'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...826..106G' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/826/2/106'
        the_whills.at[index, 'Notes'] += ' Gabanyi+ do not detect a source in this system but Muller-Sanchez+ do detect two components.'
        the_whills.at[index, 'Name'] = 'J102325.57+324348.4'


# Verified that this matching process works


In [ ]:
#the_whills

In [ ]:
# This is for Nevin+2016
nevin2016 = ((Table.read('Tables/nevin2016/table1.dat', readme = 'Tables/nevin2016/ReadMe', format='ascii.cds')).to_pandas())

ogobjs = ['J101835.77+512753.1'] # Nevin+2016 has is listed as J101835.77+512753.0
for index, row in the_whills.iterrows():
    if row['Name'] in ogobjs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Nevin+2016'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...832...67N' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/832/1/67'
        the_whills.at[index, 'Notes']+=' Nevin+ classifies this system as Rotationally domination + disturbance'

# Verified that this matching process works

# ***********
# We probably should go back and just add the bib info for any other matching objects between Nevin+ and the main \
# tables. 
# ***********


In [ ]:
#nevin2016

In [ ]:
# This is for Liu+2018
#
# CONSIDER GOING BACK AND ADDING IN INFO ON WHICH ARE UNRESOLVE AND WHICH LIKELY HAVE JETS
#

# First the galaxies detected at 8.4 GHz
liu2010obs = ['J091201.68+532036.6','J113721.36+612001.2','J124358.36-005845.4','J135251.22+654113.2','J231051.95-090011.9','J233313.17+004911.8']
#print(len(liu2010obs))
liu2010obs = pd.DataFrame(data=liu2010obs,columns=['SDSS'])
name_to_coords(liu2010obs,liu2010obs['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = liu2010obs['RA'], dec = liu2010obs['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
liu2010obs['RA1_deg'] = coordconvert.ra.degree
liu2010obs['Dec1_deg'] = coordconvert.dec.degree


tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,liu2010obs,5)
for index, row in tmatches.iterrows():
    if row['Table_flag']!='Table2':
        tmatches.at[index, 'Paper(s)'] += ' ; Liu+2018'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2018ApJ...854..169L' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aaab47'
        tmatches.at[index, 'Notes'] += 'Detected in follow-up 8.4 GHz VLBA imaging but unambiguous sub-kpc dual AGNs are not identified.'

# Now clipping out all Liu+2018 rows from the matches table
tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)

# Concatenating everything together to generate a master table here
the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
the_whills.drop(labels=['index','Table_flag'], axis=1, inplace=True) #'level_0',

###for index, row in the_whills.iterrows():
###    if row['Name'] in liu2010objs:
###        print('True1')
###        #the_whills.at[index, 'Paper(s)'] += ' ; Liu+2018 '
###        #the_whills.at[index, 'BibCode(s)'] += ' ; 2018ApJ...854..169L' 
###        #the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aaab47 ; '
###        #the_whills.at[index, 'Notes']='Detected in follow-up 8.4 GHz VLBA imaging but unambiguous sub-kpc dual AGNs are not identified.'
###

# Next the galaxies NOT detected at 8.4 GHz
liu2010obs = ['J000911.58-003654.7','J073849.75+315611.9','J080337.32+392633.1','J085841.76+104122.1','J110851.04+065901.4','J135646.11+102609.1','J171544.05+600835.7']
#print(len(liu2010obs))
liu2010obs = pd.DataFrame(data=liu2010obs,columns=['SDSS'])
name_to_coords(liu2010obs,liu2010obs['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = liu2010obs['RA'], dec = liu2010obs['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
liu2010obs['RA1_deg'] = coordconvert.ra.degree
liu2010obs['Dec1_deg'] = coordconvert.dec.degree


tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,liu2010obs,5)
for index, row in tmatches.iterrows():
    if row['Table_flag']!='Table2':
        tmatches.at[index, 'Paper(s)'] += ' ; Liu+2018'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2018ApJ...854..169L' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aaab47'
        tmatches.at[index, 'Notes'] += 'Detected in follow-up 8.4 GHz VLBA imaging but unambiguous sub-kpc dual AGNs are not identified.'

#print(len(tmatches))
# Now clipping out all Liu+2018 rows from the matches table
tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)

# Concatenating everything together to generate a master table here
the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
the_whills.drop(labels=['index','Table_flag'], axis=1, inplace=True) #'level_0',

#for index, row in the_whills.iterrows():
#    if row['Name'] in liu2010objs:
#        print('True2')
#        #the_whills.at[index, 'Paper(s)'] += ' ; Liu+2018 '
#        #the_whills.at[index, 'BibCode(s)'] += ' ; 2018ApJ...854..169L' 
#        #the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aaab47 ; '
#        #the_whills.at[index, 'Notes']='Not detected at 8.4 GHz in follow-up VLBA imaging.'

#the_whills


# Verified that this matching process works. Everything seems okay. 


In [ ]:
# This is for Shi+2014

# I created this catalog manually since Shi+2014 did not publish their catalog separately on CDS
shi2014 = pd.read_csv('Tables/Shi2014/shi2014.csv', sep=',')

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(shi2014,shi2014['Designation'])

shi2014['Name'] = shi2014['Designation']
shi2014['Name2'] = shi2014['Designation']
shi2014['z1'] = shi2014['z']
shi2014['z2'] = shi2014['z']
shi2014['z1_type'] = "spec"
shi2014['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(shi2014,shi2014['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = shi2014['RA'], dec = shi2014['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
shi2014['RA1_deg'] = coordconvert.ra.degree
shi2014['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
shi2014['RA2'] = shi2014['RA']
shi2014['Dec2'] = shi2014['Dec']

shi2014['RA2_deg'] = shi2014['RA1_deg']
shi2014['Dec2_deg'] = shi2014['Dec1_deg']

# Adding details about the coordinates
shi2014['Equinox'] = "J2000"
shi2014['Coordinate_waveband'] = "Optical"
shi2014['Coordinate_Source'] = "LAMOST"

shi2014['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
shi2014['Brightness1'] = shi2014['r']
shi2014['Brightness_band1'] = "LAMOST r-band"
shi2014['Brightness_type1'] = "mag"

shi2014['Brightness2'] = shi2014['r']
shi2014['Brightness_band2'] = "LAMOST r-band"
shi2014['Brightness_type2'] = "mag"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
shi2014['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#*******
#******************the cosmo arcsec to kpc command needs to be fixed! It is deprecated apparently!
#shi2014['Sep(kpc)'] = shi2014['Sep']*((cosmo.arcsec_per_kpc_proper(shi2014['z']))**(-1))
#************
#*******


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#shi2014['delta_z'] = shi2014['z']-shi2014['z2']
shi2014['dV'] = (2.99e+5)*((1+shi2014['z'])**2 - (1+shi2014['z2'])**2)/((1+shi2014['z'])**2+(1+shi2014['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
shi2014['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
shi2014['Confirmation Method'] = "-99"
shi2014['Paper(s)'] = "Shi+2014"
shi2014['BibCode(s)'] = "2014RAA....14.1234S"
shi2014['DOI(s)'] = "https://doi:10.1088/1674-4527/14/10/003"
shi2014['Notes'] = ''

#shi2014


In [ ]:
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,shi2014,15)

print(len(tmatches))
# There are 4 matches up to this point between the Shi+2014 catalog and the_whills

#tmatches
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Paper(s)']!='Shi+2014':
        tmatches.at[index, 'Paper(s)'] += ' ; Shi+2014'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2014RAA....14.1234S' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi:10.1088/1674-4527/14/10/003'
        tmatches.at[index, 'Notes'] += ' Shi+ also selected this in LAMOST.'
        
# Now clipping out all Comerford+2013 rows from the matches table
tmatches = tmatches[tmatches['Paper(s)']!='Shi+2014'].reset_index(drop=True)

# Concatenating everything together to generate a master table here
the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
#the_whills.drop(labels=['level_0','index'], axis=1, inplace=True)

# verified that this matching process works and that we get all of the matches we should be getting out to 15''!

#the_whills

In [ ]:
len(the_whills)

In [ ]:
#tmatches

In [ ]:
# This is for McGurk+2015

mcgurk2015 = pd.read_csv('Tables/McGurk2015/McGurk2015.csv', sep=',')
mcgurk2015t2 = pd.read_csv('Tables/McGurk2015/McGurk2015t2.csv', sep=',')
mcgurk2015t3 = pd.read_csv('Tables/McGurk2015/McGurk2015t3.csv', sep=',')
mcgurk2015t4 = pd.read_csv('Tables/McGurk2015/McGurk2015t4.csv', sep=',')


In [ ]:
def find_matches(names):
    name_dict = {}
    matches = []

    for name in names:
        first_6_chars = name[:7]
        if first_6_chars in name_dict:
            matches.append((name_dict[first_6_chars], name))
        else:
            name_dict[first_6_chars] = name

    return matches

#names = mcgurk2015t4['Name'].to_list()
#matched_pairs = find_matches(names)
#
#for pair in matched_pairs:
#    print(f"Match found: {pair[0]} and {pair[1]}")
    

In [ ]:
names = the_whills['Name'].to_list()
matched_pairs = find_matches(names)

for pair in matched_pairs:
    print(f"Match found: {pair[0]} and {pair[1]}")



In [ ]:
#mcgurk2015

In [ ]:
# Converting the coordinates
coordconvert = SkyCoord(ra = mcgurk2015t2['RA'], dec = mcgurk2015t2['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
mcgurk2015t2['RA1_deg'] = coordconvert.ra.degree
mcgurk2015t2['Dec1_deg'] = coordconvert.dec.degree


#print(len(mcgurk2015t2))
#mcgurk2015t2

In [ ]:
# Converting the coordinates
coordconvert = SkyCoord(ra = mcgurk2015t3['RA'], dec = mcgurk2015t3['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
mcgurk2015t3['RA1_deg'] = coordconvert.ra.degree
mcgurk2015t3['Dec1_deg'] = coordconvert.dec.degree

#len(mcgurk2015t3)

In [ ]:
# Converting the coordinates
coordconvert = SkyCoord(ra = mcgurk2015t4['RA'], dec = mcgurk2015t4['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
mcgurk2015t4['RA1_deg'] = coordconvert.ra.degree
mcgurk2015t4['Dec1_deg'] = coordconvert.dec.degree


#len(mcgurk2015t4)

In [ ]:
# Now here we'll be matching the tables from McGurk to the overarching catalogs

the_whills.reset_index(drop=True, inplace=True)

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,mcgurk2015t2,10)

print(len(tmatches))


# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; McGurk+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015ApJ...811...14M' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/811/1/14'
        the_whills.at[index, 'Notes'] += ' Companion within 3 arcseconds.'
        
## Now clipping out all Comerford+2013 rows from the matches table
#tmatches = tmatches[tmatches['Paper(s)']!='Shi+2014'].reset_index(drop=True)
#
## Concatenating everything together to generate a master table here
#the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
##the_whills.drop(labels=['level_0','index'], axis=1, inplace=True)


# we have to go out to 10'' in match tolerance \
# to get the last object (the rest match within 5'')

# I stripped the duplicate rows out of the McGurk table (and just add additional info to a main row)

# and I have now visually verified that we have the correct number of matches and that the \
# correct objects have been matched


In [ ]:
# Now here we'll be matching the tables from McGurk to the overarching catalogs

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,mcgurk2015t3,13)

print(len(tmatches))

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; McGurk+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015ApJ...811...14M' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/811/1/14'
        the_whills.at[index, 'Notes'] += ' No companion within 3 arcseconds.'


# I've visually checked all of the matches since we have to go out to 13'' in match \
# tolerance to get the last object 
# verified that this process works


In [ ]:
# Now here we'll be matching the tables from McGurk to the overarching catalogs

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,mcgurk2015t4,5)

#len(tmatches)

# These all match within 5'', just need to verify that the matching process works at the end

for i, j in zip(idx1, idx2):
    #print("i:", i, "j:", j)
    #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
    #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
    the_whills.at[i, 'Sep'] = mcgurk2015t4.at[j, 'NIRC2sep(as)']
    the_whills.at[i, 'dV'] = mcgurk2015t4.at[j, 'dV[OIII]']
    the_whills.at[i, 'Notes'] += mcgurk2015t4.at[j, 'Notes']

# verified that this matching process works properly


In [ ]:
# Now loading in the information from Muller-Sanchez+2015:

ms2015t1 = pd.read_csv('Tables/Mullersanchez2015/Mullersanchezt1.csv')
ms2015t7 = pd.read_csv('Tables/Mullersanchez2015/Mullersanchezt7.csv')

ms2015 = pd.concat([ms2015t1,ms2015t7], axis=1)
ms2015t4 = pd.read_csv('Tables/Mullersanchez2015/Mullersanchezt4.csv') # These are the confirmed dual AGNs
#ms2015t5 = pd.read_csv('Tables/Mullersanchez2015/Mullersanchezt5.csv') # These are the ambiguous cases

# Converting the coordinates
coordconvert = SkyCoord(ra = ms2015['RA'], dec = ms2015['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
ms2015['RA1_deg'] = coordconvert.ra.degree
ms2015['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
ms2015['RA2'] = ms2015['RA']
ms2015['Dec2'] = ms2015['Dec']

ms2015['RA2_deg'] = ms2015['RA1_deg']
ms2015['Dec2_deg'] = ms2015['Dec1_deg']


#ms2015['Confirmation Method'] = "-99"
ms2015['Paper(s)'] = "Muller-Sanchez+2015"
ms2015['BibCode(s)'] = "2015ApJ...813..103M"
ms2015['DOI(s)'] = "https://doi.org/10.1088/0004-637X/813/2/103"

#ms2015


In [ ]:
# Here's where we'll be matching the ms2015 tables back against the the_whills and filling in information as 

ms2015temp = ms2015[ms2015['Origin_Double_Peaks']=='radio jet-driven outflow'].reset_index(drop=True)
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,ms2015temp,5)
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Muller-Sanchez+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015ApJ...813..103M' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/813/2/103'
        the_whills.at[index, 'System Type'] = 'Likely Single AGN'
        the_whills.at[index, 'Notes'] += 'Muller-Sanchez+2015 find that the origin of the double-peaked lines is likely an jet-driven outflow. '
        #tmatches.at[index, 'Sep'] =
#print(len(tmatches))
#tmatches

#for i in the_whills['Paper(s)']:
#    if 'Muller-Sanchez+2015' in i:
#        print('True')

ms2015temp = ms2015[ms2015['Origin_Double_Peaks']=='rotating disk'].reset_index(drop=True)
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,ms2015temp,5)
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Muller-Sanchez+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015ApJ...813..103M' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/813/2/103'
        the_whills.at[index, 'System Type'] = 'Likely Single AGN'
        the_whills.at[index, 'Notes'] += 'Muller-Sanchez+2015 find that the origin of the double-peaked lines is likely a rotating disk.'
        #tmatches.at[index, 'Sep'] =

#print(len(tmatches))
#for i in the_whills['Paper(s)']:
#    if 'Muller-Sanchez+2015' in i:
#        print('True')

ms2015temp = ms2015[ms2015['Origin_Double_Peaks']=='AGN-driven outflow'].reset_index(drop=True)
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,ms2015temp,15)
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Muller-Sanchez+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015ApJ...813..103M' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/813/2/103'
        the_whills.at[index, 'System Type'] = 'Likely Single AGN'
        the_whills.at[index, 'Notes'] += 'Muller-Sanchez+2015 find that the origin of the double-peaked lines is likely an AGN driven outflow.'
        #tmatches.at[index, 'Sep'] =
# we have to go up to 15'' match tolerance to grab the missing object
#for i in the_whills['Paper(s)']:
#    if 'Muller-Sanchez+2015' in i:
#        print('True')       

ms2015temp = ms2015[ms2015['Origin_Double_Peaks']=='dual AGNs'].reset_index(drop=True)
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,ms2015temp,10)
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Muller-Sanchez+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015ApJ...813..103M' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/813/2/103'
        the_whills.at[index, 'System Type'] = 'Dual AGN'
        the_whills.at[index, 'Notes'] += ' '

#for i in the_whills['Paper(s)']:
#    if 'Muller-Sanchez+2015' in i:
#        print('True')  

ms2015temp = ms2015[ms2015['Origin_Double_Peaks']=='ambiguous'].reset_index(drop=True)
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,ms2015temp,5)
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if index in idx1:
        the_whills.at[index, 'Paper(s)'] += ' ; Muller-Sanchez+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015ApJ...813..103M' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/813/2/103'
        the_whills.at[index, 'Notes'] += 'Muller-Sanchez+2015 find that the origin of the double-peaked emission is still ambiguous.'

#for i in the_whills['Paper(s)']:
#    if 'Muller-Sanchez+2015' in i:
#        print('True')  


# verified that all of this matching now works!


In [ ]:
# Here w're just adding in some bib info and notes from Villforth+2015

#vill2015 = ['J095207.6+255257','J115106.7+471157','J150243.1+111557','J171544.0+600835'] --> this Villforth's origina listing
vill2015 = ['J095207.62+255257.2','J115106.69+471157.7','J150243.09+111557.3',] #--> these are the matching objects from the original tables
for index, row in the_whills.iterrows():
    if row['Name'] in vill2015:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Villforth+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015AJ....149...92V' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-6256/149/3/92'
        the_whills.at[index, 'Notes']+=' Villforth+ report clear merger features. Broad predomimantly blue [O III] wings observed. These high velocity outflows are observed only in the campact <5 kpc central regions. Given the speeds the outflows are likely driven by the AGN(s). The galaxies are redder in their central regions.The NLR is extended on sizes up to 50-100 kpc. Clear clumps in the NLRs are apparent.'


vill2015 = ['J171544.05+600835.7']
for index, row in the_whills.iterrows():
    if row['Name'] in vill2015:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Villforth+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015AJ....149...92V' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-6256/149/3/92'
        the_whills.at[index, 'Notes']+='Villorth+ report that there are no strong merger features in this system. They rule out that the dual AGN dentified by Comerford+ is the cause of the double-peaked emissioon lines.'

# Verified that the matching process here works


In [ ]:
the_whills.to_csv('the_whills_beforeLyu2016.csv', sep=',')

In [ ]:
# Loading in the catalog from Lyu+2016 now...
lyu2016 = ((Table.read('Tables/lyu2016/table1.dat', readme = 'Tables/lyu2016/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])
# Note here that any cells containing '--' in table are going to be replaced by 0's

lyu2016['Name'] = lyu2016['SDSS']
lyu2016['Name2'] = lyu2016['SDSS']
lyu2016['z2'] = lyu2016['z']
lyu2016['z1_type'] = "spec"
lyu2016['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(lyu2016,lyu2016['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = lyu2016['RA'], dec = lyu2016['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
lyu2016['RA1_deg'] = coordconvert.ra.degree
lyu2016['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
lyu2016['RA2'] = lyu2016['RA']
lyu2016['Dec2'] = lyu2016['Dec']

lyu2016['RA2_deg'] = lyu2016['RA1_deg']
lyu2016['Dec2_deg'] = lyu2016['Dec1_deg']

# Adding details about the coordinates
lyu2016['Equinox'] = "J2000"
lyu2016['Coordinate_waveband'] = "Optical"
lyu2016['Coordinate_Source'] = "SDSS"

lyu2016['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
lyu2016['Brightness1'] = -100
lyu2016['Brightness_band1'] = -100
lyu2016['Brightness_type1'] = -100

lyu2016['Brightness2'] = -100
lyu2016['Brightness_band2'] = -100
lyu2016['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
lyu2016['Sep'] = 2 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 2'' diameter of the SDSS \
# **BOSS** fiber as an upper limit

#lyu2016['Sep(kpc)'] = lyu2016['Sep']*((cosmo.arcsec_per_kpc_proper(lyu2016['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#lyu2016['delta_z'] = lyu2016['z']-lyu2016['z2']
lyu2016['dV'] = (2.99e+5)*((1+lyu2016['z'])**2 - (1+lyu2016['z2'])**2)/((1+lyu2016['z'])**2+(1+lyu2016['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
lyu2016['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
lyu2016['Confirmation Method'] = "-99"
lyu2016['Paper(s)'] = "Lyu+2016"
lyu2016['BibCode(s)'] = "2016MNRAS.463...24L"
lyu2016['DOI(s)'] = "https://doi.org/10.1093/mnras/stw1945"

lyu2016['Notes'] = ''

lyu2016.reset_index(drop=True, inplace=True)



In [ ]:
names = lyu2016['Name'].to_list()
matched_pairs = find_matches(names)

for pair in matched_pairs:
    print(f"Match found: {pair[0]} and {pair[1]}")



In [ ]:
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,lyu2016,15)

print(len(tmatches))
# There are 2 matches up to this point between the_whills
# J023658.06+024217.9
# J131642.90+175332.5

#tmatches
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in tmatches.iterrows():
    if row['Table_flag']!='Table2':
        tmatches.at[index, 'Paper(s)'] += ' ; Lyu+2016'
        tmatches.at[index, 'BibCode(s)'] += ' ; 2016MNRAS.463...24L' 
        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stw1945'
        tmatches.at[index, 'Notes'] += ' Lyu+2016 also selected this object with BOSS.'
        
# Now clipping out all Comerford+2013 rows from the matches table
tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)
#
## Concatenating everything together to generate a master table here
the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
#the_whills.drop(labels=['level_0','index'], axis=1, inplace=True)

# this matching process 'works' but the two matches end up making duplicate entries for some reason
# I do not understand why the matching process is not working properly here, where it worked perfectly for every other
# cell previously

# this cell is verified now

#the_whills

In [ ]:
# here I'm adding some code to manually strip out the duplicate two rows

rows_to_drop = the_whills[(the_whills['Name'] == 'J023658.06+024217.9') & (the_whills['Paper(s)'] == 'Shi+2014')].index

# Drop the rows
the_whills.drop(rows_to_drop, inplace=True)

rows_to_drop = the_whills[(the_whills['Name'] == 'J131642.90+175332.5') & (the_whills['Paper(s)'] == 'Smith+2010')].index

# Drop the rows
the_whills.drop(rows_to_drop, inplace=True)

the_whills.reset_index(drop=True, inplace=True)
# this cell is verified and removes the unusual duplicates from before


In [ ]:
the_whills.to_csv('the_whills_afterLyu_beforeyuan.csv', sep=',')


In [ ]:
def find_matches(names):
    name_dict = {}
    matches = []

    for name in names:
        first_6_chars = name[:7]
        if first_6_chars in name_dict:
            matches.append((name_dict[first_6_chars], name))
        else:
            name_dict[first_6_chars] = name

    return matches

names = the_whills['Name'].to_list()
matched_pairs = find_matches(names)

for pair in matched_pairs:
    print(f"Match found: {pair[0]} and {pair[1]}")
    

In [ ]:
# from astropy import units as u
# from astropy.coordinates import SkyCoord

# def angular_separation_arcsec(coord1, coord2):
#     """
#     Calculate angular separation between two sexagesimal coordinates.
    
#     Parameters:
#     coord1, coord2: tuples containing RA and Dec in the format ('13:52:07.73', '+05:25:55.8')
    
#     Returns:
#     Angular separation in arcseconds.
#     """
#     c1 = SkyCoord(coord1[0], coord1[1], unit=(u.hourangle, u.deg))
#     c2 = SkyCoord(coord2[0], coord2[1], unit=(u.hourangle, u.deg))
#     sep = c1.separation(c2)
#     return sep.arcsecond

# # Example usage:
# coord1 = ('13:52:07.0', '+05:25:55.0')
# coord2 = ('13:52:07.73', '+05:25:55.8')
# print(f"Angular separation: {angular_separation(coord1, coord2)} arcseconds")

In [ ]:
#for index, row in the_whills.iterrows():
#    if index in idx1:
#        the_whills.at[index, 'Paper(s)'] += ' ; Muller-Sanchez+2015'
#        the_whills.at[index, 'BibCode(s)'] += ' ; 2015ApJ...813..103M' 
#        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/0004-637X/813/2/103'
#        the_whills.at[index, 'System Type'] = 'Likely Single AGN'
#        the_whills.at[index, 'Notes'] += 'Muller-Sanchez+2015 find that the origin of the double-peaked lines is likely a rotating disk.'
#        #tmatches.at[index, 'Sep'] =
#


In [ ]:
#the_whills = pd.read_csv('the_whills_afterLyu_beforeyuan.csv', sep=',')


In [ ]:
# This is for Yuan+2016 (and by extension, the Reyes+ and Zakamska+ tables)
from astropy.io import ascii

yuan2016t1 = (ascii.read('Tables/yuan2016/Type2s.tbl').to_pandas())#.drop(columns=['---'])
yuan2016t2 = (ascii.read('Tables/yuan2016/Type2s_kinematics.tbl').to_pandas())#.drop(columns=['---'])

yuan2016t2 = pd.merge(yuan2016t1, yuan2016t2, on=['plate', 'fiber', 'mjd'], how='inner')

#yuan2016t2.loc[yuan2016t2['Unique']!='unique', 'Name'] = yuan2016['Unique']

yuan2016t3 = (ascii.read('Tables/yuan2016/Reyes_kinematics.tbl').to_pandas())#.drop(columns=['---'])

yuan2016t2 = yuan2016t2[yuan2016t2['dp']!=0]
print(len(yuan2016t2['unique']!='unique'))
yuan2016t3 = yuan2016t3[yuan2016t3['dp']!=0]
#I need to go through and somehow get the SDSS BOSS designations

yuan2016t3 = yuan2016t3.reset_index(drop=True)
yuan2016t2 = yuan2016t2.reset_index(drop=True)

print(len(yuan2016t2))
print(len(yuan2016t3))

yuan2016 = pd.concat([yuan2016t2,yuan2016t3])

# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=yuan2016['ra']*u.degree, dec=yuan2016['dec']*u.degree, frame='icrs')
yuan2016['ra_sexagesimal'] = coords.ra.to_string(u.hour, sep=':', precision=2)
yuan2016['dec_sexagesimal'] = coords.dec.to_string(u.deg, sep=':', precision=2)

# Convert RA and Dec to the desired format
ra_format = coords.ra.to_string(unit=u.hour, sep='', precision=2, pad=True)
dec_format = coords.dec.to_string(unit=u.deg, sep='', precision=2, alwayssign=True, pad=True)

# Concatenate to form J_format using a loop
yuan2016['J_format'] = ["J" + ra + dec for ra, dec in zip(ra_format, dec_format)]


# here we're droppping the duplicate columns from yuan's tables:
yuan2016.drop_duplicates(subset='J_format', keep='last', inplace=True)



In [ ]:
yuan2016['Name'] = yuan2016['J_format'] # str(yuan2016['plate']+yuan2016['fiber']+yuan2016['mjd'])
yuan2016['Name2'] = yuan2016['J_format'] # str(yuan2016['plate']+yuan2016['fiber']+yuan2016['mjd'])
yuan2016['z2'] = yuan2016['z']
yuan2016['z1_type'] = "spec"
yuan2016['z2_type'] = "spec"

# Converting the coordinates
#coordconvert = SkyCoord(ra = yuan2016['RA'], dec = yuan2016['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
yuan2016['RA1_deg'] = yuan2016['ra']#coordconvert.ra.degree
yuan2016['Dec1_deg'] = yuan2016['dec']#coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
yuan2016['RA2'] = yuan2016['ra']
yuan2016['Dec2'] = yuan2016['dec']

yuan2016['RA2_deg'] = yuan2016['RA1_deg']
yuan2016['Dec2_deg'] = yuan2016['Dec1_deg']

# Adding details about the coordinates
yuan2016['Equinox'] = "J2000"
yuan2016['Coordinate_waveband'] = "Optical"
yuan2016['Coordinate_Source'] = "SDSS"

yuan2016['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
yuan2016['Brightness1'] = -100
yuan2016['Brightness_band1'] = -100
yuan2016['Brightness_type1'] = -100

yuan2016['Brightness2'] = -100
yuan2016['Brightness_band2'] = -100
yuan2016['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
yuan2016['Sep'] = 2 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 2'' diameter of the SDSS \
# **BOSS** fiber as an upper limit

#yuan2016['Sep(kpc)'] = yuan2016['Sep']*((cosmo.arcsec_per_kpc_proper(yuan2016['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#yuan2016['delta_z'] = yuan2016['z']-yuan2016['z2']
yuan2016['dV'] = (2.99e+5)*((1+yuan2016['z'])**2 - (1+yuan2016['z2'])**2)/((1+yuan2016['z'])**2+(1+yuan2016['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
yuan2016['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
yuan2016['Confirmation Method'] = "-99"
yuan2016['Paper(s)'] = "Yuan+2016"
yuan2016['BibCode(s)'] = "2016MNRAS.462.1603Y"
yuan2016['DOI(s)'] = "https://doi.org/10.1093/mnras/stw1747"

# Make sure to go back and add the citations for the papers they ask to be cited in addition to Yuan+.

yuan2016['Notes'] = ''

# here we're dropping some duplicates that weren't exactly identical in Yuan's tables
rows_to_drop = yuan2016[(yuan2016['Name'] == 'J010607.06+000927.47') | (yuan2016['Name'] == 'J090247.04+012028.39') | (yuan2016['Name'] == 'J090247.04+012028.39') | (yuan2016['Name'] == 'J092152.56+515348.12') | (yuan2016['Name'] == 'J112319.20+302825.32')].index

# Drop the rows
yuan2016.drop(rows_to_drop, inplace=True)
yuan2016.reset_index(drop='True', inplace=True)
# verified that this drops the unusual duplicates


#yuan2016


In [ ]:
names = yuan2016['Name'].to_list()
matched_pairs = find_matches(names)

for pair in matched_pairs:
    print(f"Match found: {pair[0]} and {pair[1]}")




In [ ]:
#yuan2016.to_csv('yuantest.csv', sep=',')

In [ ]:
#len(the_whills['Name'])

In [ ]:
#the_whills.reset_index(drop=True, inplace=True)
#yuan2016.reset_index(drop=True, inplace=True)


In [ ]:
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,yuan2016,10)

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Paper(s)'] += ' ; Yuan+2016'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2016MNRAS.462.1603Y' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stw1747'

print(len(idx2))
# dropping the matching indices from the yuan2016 table
yuan2016.drop(idx2, axis=0, inplace=True)
yuan2016.reset_index(drop=True, inplace=True)

the_whills = pd.concat([the_whills,yuan2016])



In [ ]:
#the_whills = the_whills_test

In [ ]:
#tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,yuan2016,10)
#
#print(len(idx2))



In [ ]:
names = the_whills['Name'].to_list()
matched_pairs = find_matches(names)

for pair in matched_pairs:
    print(f"Match found: {pair[0]} and {pair[1]}")




In [ ]:

# tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,yuan2016,10)

# # There are a lot of matches between Lyu+ and Yuan+
# #tmatches
# # Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
# for index, row in tmatches.iterrows():
#     if row['Paper(s)']!='Yuan+2016':
#         tmatches.at[index, 'Paper(s)'] += ' ; Yuan+2016'
#         tmatches.at[index, 'BibCode(s)'] += ' ; 2016MNRAS.462.1603Y' 
#         tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stw1747'
#         tmatches.at[index, 'Notes'] += ' Yuan+ also selected this target.'
        
# # Now clipping out all Comerford+2013 rows from the matches table
# tmatches = tmatches[tmatches['Paper(s)']!='Yuan+2016'].reset_index(drop=True)

# # Create a boolean mask where each value is True if the 'Name' in tunique is also in tmatches
# mask = tunique['Name'].isin(tmatches['Name']) #shamelessly asked chatgpt to quickly write this because I was too lazy to go find my old script
# tunique = tunique[(~mask) & (tunique['Paper(s)']=='Yuan+2016')]
# print(len(tmatches))
# print(len(tunique))

# ### Concatenating everything together to generate a master table here
# the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
# ###the_whills.drop(labels=['level_0','index'], axis=1, inplace=True)
# print(len(tmatches))

# # verified that this now appears to be functioning properly
# # I had to add some code that strips out objects from tunique that are actually within tmatches...


In [ ]:
# names = the_whills['Name'].to_list()
# matched_pairs = find_matches(names)

# for pair in matched_pairs:
#     print(f"Match found: {pair[0]} and {pair[1]}")



In [ ]:
# the_whills['Name'].to_list()

In [ ]:
# the_whills['Name'].to_csv('check_yuan_dups.csv', index=False)


In [ ]:
# len(the_whills['Name'])

In [ ]:
# def find_matches(names):
#     name_dict = {}
#     matches = []

#     for name in names:
#         first_6_chars = name[:7]
#         if first_6_chars in name_dict:
#             matches.append((name_dict[first_6_chars], name))
#         else:
#             name_dict[first_6_chars] = name

#     return matches

# names = the_whills['Name'].to_list()
# matched_pairs = find_matches(names)

# for pair in matched_pairs:
#     print(f"Match found: {pair[0]} and {pair[1]}")
    

In [ ]:
# Here we're adding the citation information and relevant designations/other info from individual papers that \ 
# have discuss the double-peaked sources

# First Liu+2018  (dual AGN candidate in a dwarf galaxy)
liu2018obs = ['J092455.24+051052.0']
for index, row in the_whills.iterrows():
    if row['Name'] in liu2018obs:
        #print('True')
        the_whills.at[index, 'dV'] = 437 #km s^-1 --> This is the from the OIII lines but is consistent with the Hbeta lines within the uncertainties 
        the_whills.at[index, 'Sep'] = 0.4
        the_whills.at[index, 'Paper(s)'] += ' ; Liu+2018 '
        the_whills.at[index, 'BibCode(s)'] += ' ; 2018ApJ...862...29L' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aac9cb'
        the_whills.at[index, 'Notes'] += 'Liu+ resolve two stellar bulges and two NLRs in this system using HST. The HST data disfavor a biconical outflow scenario and they favor a dual AGN scenario; given the separation they cannot rule out a single AGN ionizing both nuclei. Liu+ finds the separation to be slightly smaller for the OIII emitting regions than the host stellar bulges.'

# Now Severgnini+2018 and Liu+2020
severobs = ['J085512+642345']
for index, row in the_whills.iterrows():
    if row['Name'] in severobs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Severgnini+2018 ; Liu+2020'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2018MNRAS.479.3804S ; 2020ApJ...896..122L' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aaab47 ; https://doi.org/10.3847/1538-4357/ab952d '
        the_whills.at[index, 'Notes'] += 'Severgnini rule out strong radio jets but cannot rule out faint radio jets. They note that the velocity offsets are fully consistent with the rotation velocities measured in nearby galaxies and therefore the offsets could be explained by kinematics in a single NLR. They detect X-ray flux variability (periodicity of ~25 months) and argue it is intrinsic flux variations rather than varying obscuration. They also report a tentive double-peaked iron line due either to rational in an accretiondisk or a binary AGN. Under the binary hypothesis the double peaked optical lines would be due to rotationin the galaxy and not the binary. By measuring the power spectrum of this object Liu+2020 rejected the 25 month periodicity that Severgnini found. Severgnini did not fit the power spectrum.'

# And now we need to add in the new information from Serafinelli+2020
for index, row in the_whills.iterrows():
    if row['Name'] in severobs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Serafinelli+2020'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2020ApJ...902...10S' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/abb3c3'
        the_whills.at[index, 'Notes'] += ' This object was excluded from the Serafinelli+ due to the g-test selection criteria. Serafinelli+ show the power spectrum density is consistent with white noise but when they perform a weighted sinusoid fit they find a period of ~26.3 months. Whenthey consider the 123 month data they find a period of 26.0 months and claim to reject the null at 3.4sigma. This period is consistent with that proposed by Severgnini+.'

# Verified that the matching process works here


In [ ]:
#print(the_whills[the_whills['Name']=='J092455.24+051052.0']['Paper(s)'])
#print(the_whills[the_whills['Name']=='J085512+642345']['Paper(s)'])


In [ ]:
#
#
# When we match/discuss J0841+0101, make sure to mention that Keel flagged that as having two AGN
# Also bring up Keel+2019 when discuss J1354, which was selected by Liu+2011
#
# Keel+2019,2019MNRAS.483.4847K,https://doi.org/10.1093/mnras/sty3332


In [ ]:
the_whills

In [ ]:
# Here adding in citations for Rubinur+ and Das+
# These papers discussed the double-peaked AGN J120320+131931 identified by Wang+09

rubobs = ['J120320+131931']
for index, row in the_whills.iterrows():
    if row['Name'] in rubobs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Rubinur+2017 ; Das+2018'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2017MNRAS.465.4772R ; 2018BSRSL..87..299D' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stw2981 ; https://ui.adsabs.harvard.edu/abs/2018BSRSL..87..299D/abstract'
        the_whills.at[index, 'Notes'] += ' Rubinur+ report distinct radio hot spots on either side of the nucleus.\
        They detect an S-shaped morphology 6 GHz as well as at 8.5 GHz and 11.5 GHz. They argue this object is\
        a CSS/CSO. They argue the observed jet precession can be explained by a binary a dual or a single AGN\
        with a warped acretinon disk. Das+ report the detection of an S-shaped radio morphology at both 8.5\
        and 11.5 GHz. They argue for a precessing jet that could be due to a binary a dual or a single AGN \
        with tilted accretion disk. It is unclear if the results discussed in Das+ are actually the results \
        of Rubinur+.'

#### EVIDENTLY MCGURK+2015 AND FU+12 ALSO LOOKED AT THIS!


# Now to add the relevant info from Rubinur+2018:
rubobs = ['J120320+131931']
for index, row in the_whills.iterrows():
    if row['Name'] in rubobs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Rubinur+2018'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2018JApA...39....8R' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1007/s12036-018-9512-y'
        the_whills.at[index, 'Notes'] += 'Rubinur+2018 resolve the two components previously reported in the radio structure of this object and discover they are hot spots of an S-shaped radio jet. They conclude this is likely a COS but do not rule out a gravitationally bound binary to explain the precession. '

rubobs = ['J161708.95+222628.0']
for index, row in the_whills.iterrows():
    if row['Name'] in rubobs:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Rubinur+2018'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2018JApA...39....8R' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1007/s12036-018-9512-y'
        the_whills.at[index, 'Notes'] = ' '
        the_whills.at[index, 'Notes'] += 'Rubinur+2018 find this is a merging system with two optical nuclei and two radio sources coincident with teh nuclei at both 6 and 15 GHz in EVLA imaging. Separation is ~5.6 kpc. Primary is consistent in optical with an AGN but secondary is consistent with SF. The radio emission in secondary is consistent with SF. '

# Rubinur+2018 do not provide any further details about their objects....

# Verified that the matching process here works


In [ ]:
#print(the_whills[the_whills['Name']=='J120320+131931']['Paper(s)'])
#print(the_whills[the_whills['Name']=='J161708.95+222628.0']['Paper(s)'])


In [ ]:
# Here we are loading in the information from Comerford+2018

# WE NEED TO GO BACK IN AND ADD THE ADDITIONAL INFORMATION FOR THE 8 PROMISING DUAL CANDIDATES NOT INCLUDED IN\
# A CDS TABLE
comerford2018t1 = ((Table.read('Tables/Comerford2018/table1.dat', readme = 'Tables/Comerford2018/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])
comerford2018t2 = ((Table.read('Tables/Comerford2018/table2.dat', readme = 'Tables/Comerford2018/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])


secondary_objs = comerford2018t2.iloc[1::2]#.reset_index(drop=True)
# Get the indices of rows in every_other_df
secondary_objs_indices = secondary_objs.index
# Drop rows from original_df
comerford2018t2 = comerford2018t2.drop(secondary_objs_indices).reset_index(drop=True)
secondary_objs = secondary_objs.reset_index(drop=True)
# Renaming columns for the secondary objects
secondary_objs.rename(columns={'Name':'Name_2', 'PAobs':'PAobs_2', 'NR':'NR_2', 'Vr':'Vr_2', 'E_Vr':'E_Vr_2',\
                               'e_Vr':'e_Vr_2', 'sigma1':'sigma1_2', 'E_sigma1':'E_sigma1_2',\
                               'e_sigma1':'e_sigma1_2', 'sigma2':'sigma2_2', 'E_sigma2':'E_sigma2_2',\
                               'e_sigma2':'e_sigma2_2', 'PAgal':'PAgal_2', 'PAO3':'PAO3_2', 'e_PAO3':'e_PAO3_2',\
                               'A':'A_2', 'Class':'Class_2'}, inplace=True)
comerford2018t2.rename(columns={'Name':'Name_1', 'PAobs':'PAobs_1', 'NR':'NR_1', 'Vr':'Vr_1', 'E_Vr':'E_Vr_1',\
                         'e_Vr':'e_Vr_1', 'sigma1':'sigma1_1', 'E_sigma1':'E_sigma1_1', 'e_sigma1':'e_sigma1_1',\
                         'sigma2':'sigma2_1', 'E_sigma2':'E_sigma2_1', 'e_sigma2':'e_sigma2_1', 'PAgal':'PAgal_1',\
                         'PAO3':'PAO3_1', 'e_PAO3':'e_PAO3_1', 'A':'A_1', 'Class':'Class_1'}, inplace=True)

comerford2018t2 = pd.concat([comerford2018t2,secondary_objs], axis=1).reset_index(drop=True)
comerford2018 = pd.concat([comerford2018t1,comerford2018t2], axis=1).reset_index(drop=True)



comerford2018['Name'] = comerford2018['SDSS']
comerford2018['Name2'] = comerford2018['SDSS']
#comerford2018['z1'] = comerford2018['z']
#comerford2018['z2'] = comerford2018['z']
#comerford2018['z1_type'] = "spec"
#comerford2018['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(comerford2018,comerford2018['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = comerford2018['RA'], dec = comerford2018['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2018['RA1_deg'] = coordconvert.ra.degree
comerford2018['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
comerford2018['RA2'] = comerford2018['RA']
comerford2018['Dec2'] = comerford2018['Dec']

comerford2018['RA2_deg'] = comerford2018['RA1_deg']
comerford2018['Dec2_deg'] = comerford2018['Dec1_deg']

# Adding details about the coordinates
comerford2018['Equinox'] = "J2000"
comerford2018['Coordinate_waveband'] = "Optical"
comerford2018['Coordinate_Source'] = "SDSS"

comerford2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
comerford2018['Brightness1'] = "-99"
comerford2018['Brightness_band1'] = "-99"
comerford2018['Brightness_type1'] = "-99"

comerford2018['Brightness2'] = "-99"
comerford2018['Brightness_band2'] = "-99"
comerford2018['Brightness_type2'] = "-99"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
comerford2018['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#*******
#******************the cosmo arcsec to kpc command needs to be fixed! It is deprecated apparently!
#comerford2018['Sep(kpc)'] = comerford2018['Sep']*((cosmo.arcsec_per_kpc_proper(comerford2018['z']))**(-1))
#************
#*******


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#comerford2018['delta_z'] = comerford2018['z1']-comerford2018['z2']
#comerford2018['dV'] = (2.99e+5)*((1+comerford2018['z1'])**2 - (1+comerford2018['z2'])**2)/((1+comerford2018['z1'])**2+(1+comerford2018['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
comerford2018['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
comerford2018['Confirmation Method'] = "-99"
comerford2018['Paper(s)'] = "Comerford+2018"
comerford2018['BibCode(s)'] = "2018ApJ...867...66C"
comerford2018['DOI(s)'] = "https://doi.org/10.3847/1538-4357/aae2b4"

#comerford2018



In [ ]:
comerford2018t3 = pd.read_csv('Tables/Comerford2018/comerford2018t3.csv', sep=',')
comerford2018t4 = pd.read_csv('Tables/Comerford2018/comerford2018t4.csv', sep=',')

#comerford2018 = pd.concat([comerford2018,comerford2018t3], axis=1, names=['Name1','SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = comerford2018t3['RA'], dec = comerford2018t3['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2018t3['RA1_deg'] = coordconvert.ra.degree
comerford2018t3['Dec1_deg'] = coordconvert.dec.degree


#comerford2018t3

In [ ]:
# Now implementing a matching process to see how many in the Comerford+2018 sample are in the original table...

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,comerford2018,14)

# we should not need to go up 14'' to match everything, but evidently that's the only way to get it to work
# (5'' gets 180, but we should see 190 matches, 95 for the overarching table and 95 from Comerford+2018)
# Using 14'', we get everything, and I've visually verified that the target names match


# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
#for index, row in tmatches.iterrows():
#    if row['Table_flag']!='Table2':
#        tmatches.at[index, 'Paper(s)'] += ' ; Comerford+2018'
#        tmatches.at[index, 'BibCode(s)'] += ' ; 2018ApJ...867...66C' 
#        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aae2b4'
#        tmatches.at[index, 'Notes'] += ' Comerford+2018 studied the kinematics that give rise to the double-peaks.'

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Paper(s)'] += ' ; Comerford+2018'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2018ApJ...867...66C' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aae2b4'
    the_whills.at[i, 'Notes'] += ' Comerford+2018 studied the kinematics that give rise to the double-peaks.'

## Now clipping out all Comerford+2013 rows from the matches table
#tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)
##
### Concatenating everything together to generate a master table here
#the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
###the_whills.drop(labels=['level_0','index'], axis=1, inplace=True)


# verified that this process works

# *****
# BUT WE NEED TO COME BACK AND ADD IN ANGULAR SEPARATION AND OTHER INFO FROM COMERFORD'S TABLE!!!
# *****

In [ ]:
names = the_whills['Name'].to_list()
matched_pairs = find_matches(names)

for pair in matched_pairs:
    print(f"Match found: {pair[0]} and {pair[1]}")




In [ ]:
#the_whills

In [ ]:
# Now implementing a matching process to see how many in the Comerford+2018 sample are in the original table...

tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,comerford2018t3,5)

# we should not need to go up 14'' to match everything, but evidently that's the only way to get it to work
# (5'' gets 180, but we should see 190 matches, 95 for the overarching table and 95 from Comerford+2018)
# Using 14'', we get everything, and I've visually verified that the target names match

len(tmatches)

## Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
#for index, row in tmatches.iterrows():
#    if row['Table_flag']!='Table2':
#        tmatches.at[index, 'Notes'] += ' Comerford+2018 found a companion within 30 kpc that also shows optical evidence of being an AGN. This is therefore a stronger case for dual AGN but not due at all to the double-peaked emission lines.'
    
for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Notes'] += ' Comerford+2018 found a companion within 30 kpc that also shows optical evidence of being an AGN. This is therefore a stronger case for dual AGN but not due at all to the double-peaked emission lines.'


## Now clipping out all Comerford+2013 rows from the matches table
#tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)
##
### Concatenating everything together to generate a master table here
#the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)


# verified that this process works

# *****
# BUT WE NEED TO COME BACK AND ADD IN ANGULAR SEPARATION AND OTHER INFO FROM COMERFORD'S TABLE!!!
# *****

In [ ]:
#print(tmatches['Name'])#.to_list()

# J000656.85+154847.9 missing in matches but it is in the the_whills... not sure why we can't match
# and it superficially looks like we got all matches because J112659.54+294442.8 gets matches twice for some reason


In [ ]:
# This is for Wang 2019

# Wang+2019 did not publish their catalog in their paper or as a separate CDS catalog. I will request it for \
# inclusion in the next veraion but I am not delaying the curent version for that.




In [ ]:
# Here we're loading in RUbinur+2019
# Rubinur+2019 did NOT include the previously used designations from the prior works and instead used IAU \
# designations. For this reason we'll need to match by RA and Dec

rubinur2019 = pd.read_csv('Tables/Rubinur2019/Rubinur2019.csv', sep=',')


coordconvert = SkyCoord(ra = rubinur2019['RA'], dec = rubinur2019['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
rubinur2019['RA1_deg'] = coordconvert.ra.degree
rubinur2019['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
rubinur2019['RA2'] = rubinur2019['RA']
rubinur2019['Dec2'] = rubinur2019['Dec']

rubinur2019['RA2_deg'] = rubinur2019['RA1_deg']
rubinur2019['Dec2_deg'] = rubinur2019['Dec1_deg']


#rubinur2019['Confirmation Method'] = "-99"
rubinur2019['Paper(s)'] = "Rubinur+2019"
rubinur2019['BibCode(s)'] = "2019MNRAS.484.4933R"
rubinur2019['DOI(s)'] = "https://doi.org/10.1093/mnras/stz334"

##rubobs = ['J120320+131931']
#for index, row in the_whills.iterrows():
#    if row['Name'] in rubobs:
#        #print('True')
#        the_whills.at[index, 'Paper(s)'] += ' ; Rubinur+2018 '
#        the_whills.at[index, 'BibCode(s)'] += ' ; 2018JApA...39....8R ' 
#        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1007/s12036-018-9512-y'
#        the_whills.at[index, 'Notes'] += 'Rubinur+2018 resolve the two components previously reported in the radio structure of this object and discover they are hot spots of an S-shaped radio jet. They conclude this is likely a COS but do not rule out a gravitationally bound binary to explain the precession. '
#

# Rubinur+ reports on two cmpact radio sources in the following systems:
# SDSS J100602.13+071130.9 --> They argue this is a dual based on the two radio sources and the BPT line ratios 
# SDSS J135558.08+001530.6 --> dual AGN candidate but the second core does not coincide with an optical nucleus. Swcond nucleus might be SF.
# 2MASX J16170895+2226279 --> dual candidate

# Z-shaped radio morphology here: SDSS J110215.68+290725.2
# Two steep spectrum components in this system: 2MASX J14454130+3341080. Could be dual or a single core-jet
# Two X-ray sources  in Chandra but only one radio source in  2MASX J09120164+5320369

# The remainder were single detecteds, or in the case  of 2MASX J15001769+1051100, not detected.

# Rubinur+ did not include distinct coordinates to the radio sources 

rubinur2019



In [ ]:
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,rubinur2019,10)

# This now matches all objects from Ge+12 that I'm including
# Three objects from Rubinur were emphatically NOT double-peaked AGNs despite them claiming to select double-\
# peaked AGNs from Ge+. One was a SF-SF double, another was ambiguous, and the third was a Type II-SF double.

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Sep'] = rubinur2019.at[j, 'Radio_core_separation_as']
    the_whills.at[i, 'Paper(s)'] += ' ; Rubinur+2019'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2019MNRAS.484.4933R' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stz334'
    the_whills.at[i, 'Notes'] += str(rubinur2019.at[j, 'Notes'])

the_whills.loc[the_whills['Name']=='J100602.14+071131.0', 'System Type'] = 'Dual AGN'    

#len(tmatches)

#the_whills

# verified that this matching process works and the correct notes are added


In [ ]:
# Adding in the double peaked sources from Kim+2020

kim2020 = pd.read_csv('Tables/Kim2020/Kim_2020.csv', sep=',')

# THERE IS ONE OVERLAPPIING SOURCE BETWEEN THIS SAMPLE AND SMITH+2010

# MAKE SURE TO ADD THESE NOTES IN DOWN BELOW!!!!!
# Kim report double radio cores in J122313.21+540906.5 and possibly in J141041.50+223337.0
#J 080544.13+113040.2, J123557.86+582122.9, and J125016.21+045745.0 show double nuclei with 3''
# J123915.40+531414.6 shows a double optical nucleus and companion-like object to the northeast

kim2020['Name'] = kim2020['Name(SDSS)']
kim2020['Name2'] = kim2020['Name(SDSS)']
kim2020['z2'] = kim2020['z']
kim2020['z1_type'] = "spec"
kim2020['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(kim2020,kim2020['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = kim2020['RA'], dec = kim2020['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
kim2020['RA1_deg'] = coordconvert.ra.degree
kim2020['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
kim2020['RA2'] = kim2020['RA']
kim2020['Dec2'] = kim2020['Dec']

kim2020['RA2_deg'] = kim2020['RA1_deg']
kim2020['Dec2_deg'] = kim2020['Dec1_deg']

# Adding details about the coordinates
kim2020['Equinox'] = "J2000"
kim2020['Coordinate_waveband'] = "Optical"
kim2020['Coordinate_Source'] = "SDSS"

kim2020['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
kim2020['Brightness1'] = -100
kim2020['Brightness_band1'] = -100
kim2020['Brightness_type1'] = -100

kim2020['Brightness2'] = -100
kim2020['Brightness_band2'] = -100
kim2020['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
kim2020['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 2'' diameter of the SDSS \
# **BOSS** fiber as an upper limit

#kim2020['Sep(kpc)'] = kim2020['Sep']*((cosmo.arcsec_per_kpc_proper(kim2020['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#kim2020['delta_z'] = kim2020['z']-kim2020['z2']
#kim2020['dV'] = (2.99e+5)*((1+kim2020['z'])**2 - (1+kim2020['z2'])**2)/((1+kim2020['z'])**2+(1+kim2020['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
kim2020['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
kim2020['Confirmation Method'] = "-99"
kim2020['Paper(s)'] = "Kim+2020"
kim2020['BibCode(s)'] = "2020ApJ...904...23K"
kim2020['DOI(s)'] = "https://doi.org/10.3847/1538-4357/abb9a0"

kim2020['Notes'] = ''
#kim2020



In [ ]:
# here we're matching the kim2020 table with the_whills
tunique, tmatches, idx1, idx2 = match_tables_fib(the_whills,kim2020,5)

print(len(tmatches))

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
#for index, row in tmatches.iterrows():
#    if row['Table_flag']!='Table2':
#        tmatches.at[index, 'Paper(s)'] += ' ; Kim+2020'
#        tmatches.at[index, 'BibCode(s)'] += ' ; 2020ApJ...904...23K' 
#        tmatches.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/abb9a0'
#        tmatches.at[index, 'Notes'] += ' Kim+2020 reselected this based on double-peaked lines and compared to spectra from known dual AGNs. Also looked for companions.'

for i, j in zip(idx1, idx2):
    the_whills.at[i, 'Paper(s)'] += ' ; Kim+2020'
    the_whills.at[i, 'BibCode(s)'] += ' ; 2020ApJ...904...23K' 
    the_whills.at[i, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/abb9a0'
    the_whills.at[i, 'Notes'] += ' Kim+2020 reselected this based on double-peaked lines and compared to spectra from known dual AGNs. Also looked for companions.'

kim2020.drop(idx2, axis=0, inplace=True)
kim2020.reset_index(drop=True, inplace=True)

the_whills = pd.concat([the_whills,kim2020])



## Now clipping out all Comerford+2013 rows from the matches table
#tmatches = tmatches[tmatches['Table_flag']!='Table2'].reset_index(drop=True)
##
#
## now adding in code to strip out objects that are for some reason listed in tunique but should only be tmatches....
#mask = tunique['Name'].isin(tmatches['Name']) #shamelessly asked chatgpt to quickly write this because I was too lazy to go find my old script
#tunique = tunique[~mask]
#print(len(tmatches))
#print(len(tunique))
#
### Concatenating everything together to generate a master table here
#the_whills = pd.concat([tmatches,tunique]).sort_values(by='Name').reset_index(drop=True)
###the_whills.drop(labels=['level_0','index'], axis=1, inplace=True)

# 

# However, Kim+2020 claim that there should be an overlap of 8 objects between them and Smith+2010 \
# (so there should be 69 unique objects to Kim+). However, I only find 7 matches (so 70 unique candidates). \
# Even if I go out to 20'', I do not find the missing object. I think this is a typo in their manuscript, \
# as when I match the_whills and Kim+2020, and then I check for duplicate names using the first 6-7 characters \
# python finds no duplicate entries. We may want to consider reaching out. The 8th match is with Ge+2012


In [ ]:
#tmatches

In [ ]:
names = the_whills['Name'].to_list()
matched_pairs = find_matches(names)

for pair in matched_pairs:
    print(f"Match found: {pair[0]} and {pair[1]}")



In [ ]:
#tmatches

In [ ]:
#Here we're adding in the results from Foord+2020

objs = ['J014209-005049','J075223.35+273643.1','J084135.09+010156.2','J085416.76+502632\
.0','J095207.62+255257.2','J100654.20+464717.2','J112659.54+294442.8','J123915.40+531414.6',\
        'J132231.86+263159.1','J135646.11+102609.1','J144804.17+182537.9','J160436.21+500958.1']
# Foord+ listed J014209-005049 as J014209.01-005050.0
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        the_whills.at[index, 'Paper(s)'] += ' ; Foord+2020'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2020ApJ...892...29F' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/ab72fa'
        #the_whills.at[index, 'Notes']='AGNs confirmed via X-rays and reanalyses of optical spectroscopy.'
        #the_whills.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'

#the_whills.at[index, 'System Type']='Dual AGN'

objs = ['J112659.54+294442.8']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        #the_whills.at[index, 'System Type']='Dual AGN'
        the_whills.at[index, 'Notes']+=' Foord+ confirm the claim by Comerford+ that this is a dual AGN. BAYMAX favors a dual X-ray source model.'

objs = ['J084135.09+010156.2']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        the_whills.at[index, 'Notes']+=' Foord+2020 conclude that this is likely a single resolved point source.'

objs = ['J075223.35+273643.1','J144804.17+182537.9','J135646.11+102609.1']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        the_whills.at[index, 'Notes']+=' Foord+2020 find that this source has strong BF values in favor of a dual X-ray point source model.'

objs = ['J075223.35+273643.1']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        the_whills.at[index, 'Notes']+=' Specifically the dual point source model is only favored when using non-informative priors However they cannot strongly conclude that this system consistutes a dual AGN despite a strong indication in the X-rays.'

objs = ['J144804.17+182537.9']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        the_whills.at[index, 'Notes']+=' Though a dual point source model is favored with informative and non-informative priors the luminosity is below their AGN criterion and the X-ray spectrum of the secondary is very soft.'

objs = ['J014209-005049','J084135.09+010156.2','J095207.62+255257.2','J123915.40+531414.6',\
        'J132231.86+263159.1','J014209.01−005050.0','J085416.76+502632.0','J100654.20+464717.2',\
        'J160436.21+500958.1']
for index, row in the_whills.iterrows():
    if row['Name'] in objs:
        the_whills.at[index, 'Notes']+=' Foord+2020 find that this source has BF values that favor a single point source model.'


# verified that all of this matching works properly
# However, it is missing the entry for J0841+0101 because J0841+0101 does not yet exists in this table. \
# J0841+0101 was added to Comerford+'s sample in 2015 (I think)


In [ ]:
#for index, row in the_whills.iterrows():
#    if 'Foord+2020' in row['Paper(s)']:
#        print(row['Name'])

In [ ]:
# Here we're adding in just some doi information for Song+2020

for index, row in the_whills.iterrows():
    if 'Smith+2010' in row['Paper(s)']:
        #print('True')
        the_whills.at[index, 'Paper(s)'] += ' ; Song+2020 '
        the_whills.at[index, 'BibCode(s)'] += ' ; 2020MNRAS.491.4023S ' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stz3354'
        the_whills.at[index, 'Notes'] += ' Song+2020 examined a subsample of the double-peaked objects in Smith+2010 but did not include a table specifying the objects. We include a reference to all of them for now. Song+ disfavor a binary hypothesis.'

# verified that this cell works properly


In [ ]:
the_whills

In [ ]:
# formatting the table now to drop bad columns
the_whills.drop(labels=['level_0','index','SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh','MJD','Plate',\
                        'Fib','e_z','rmag','sigma','e_sigma','DelVb','e_DelVb','DelVr','e_DelVr','sigb',\
                        'e_sigb','sigr','e_sigr','FbO3','e_FbO3','FrO3','e_FrO3','FbHa','e_FbHa','FrHa',\
                        'e_FrHa','T','z1','NDWFS','delta_z','zsdss','Table_flag','Designation','r','Fiber',\
                        'zg','n_zg','FWHM[OIII]1','FWHM[OIII]2','v[OIII]1','v[OIII]2','vHb1','vHb2','EW[OIII]1',\
                        'EW[OIII]2','plate','fiber','mjd','z_x','select','ra','dec','magu','magg','magr',\
                        'magi','magz','emagu','emagg','emagr','emagi','emagz','oiiilum','oiiiflux','oiiirew',\
                        'oiiiw80','nevflux','nevrew','nevsnr','w1','w2','w3','w4','ew1','ew2','ew3','ew4',\
                        'lum5','lum12','unique','z_y','amp1','vel1','sig1','amp2','vel2','sig2','amp3','vel3',\
                        'sig3','amp4','vel4','sig4','fwhm','fwqm','w50','w80','w90','relasym','r9050','dp',\
                        'ra_sexagesimal','dec_sexagesimal','J_format','Name(SDSS)','r(mag)','Reduced_chi2',\
                        'Reduced_chi2.1','deltaV'], axis=1, inplace=True)

the_whills


In [ ]:
# renaming RA and Dec to RA1 and Dec1

the_whills.rename(columns={"RA": "RA1", "Dec": "Dec1"}, inplace=True)

#the_whills


In [ ]:
the_whills.rename(columns={"Name": "Name1"}, inplace=True)


In [ ]:
# finally saving the table now
the_whills.to_csv('Double-peaked_dual_agn_candidates.csv', sep=',', index=False)


In [ ]:
# Deprecated programs below

In [ ]:
# # Here's a function that we'll use to match catalogs together, since by this point our column names will be uniform


# def match_tables(t1,t2,match_tol):
#     # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
#     c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
#     c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
#     idx, d2d, d3d = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
#     # idx are the indices in table 2 which are the closest matching rows to table 1
    
#     # Adding a match tolerance here, with user input for the function
#     max_sep = match_tol * u.arcsec # The max match tolerance will be 5'
#     sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
    
#     # Now applying the match tolerance to the matches for each table
#     c1_matches = c1[sep_constraint] # applying to the matches in table 1
#     c1_matches = c2[idx[sep_constraint]] # applying to the matches in table 2

#     # Now storing the relevant index information for later...
#     c2_matchesRA1 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the KT catalog
#     c2_matchesRA1['idx'] = idx[sep_constraint] # Storing the indices
#     c2_matchesRA1['RA1'] = c2_matches.ra.degree # Storing the coordinates in these two rows
#     c2_matchesRA1['Dec1'] = c2_matches.dec.degree
    
#     # Next, we match RA1 and Dec1 of t1 to RA2 and Dec2 of t2, just to ensure we haven't matched to the wrong nucleus \
#     # and missed a matching system...
#     c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
#     c2 = SkyCoord(ra=t2['RA2_deg']*u.degree, dec=t2['Dec2_deg']*u.degree) # storing coordinates for table 2
#     idx, d2d, d3d = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    
#     # Adding a match tolerance again here
#     sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
    
#     # Now applying the match tolerance to the matches for each table
#     c1_matches = c1[sep_constraint] # applying to the matches in table 1
#     c1_matches = c2[idx[sep_constraint]] # applying to the matches in table 2

#     # Now storing the relevant index information for later...
#     c2_matchesRA2 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the KT catalog
#     c2_matchesRA2['idx'] = idx[sep_constraint] # Storing the indices
#     c2_matchesRA2['RA1'] = c2_matches.ra.degree # Storing the coordinates in these two rows
#     c2_matchesRA2['Dec1'] = c2_matches.dec.degree

#     # Concatenating these frames now
#     frames = [c2_matchesRA1,c2_matchesRA2]
#     c2_dups = (pd.concat(frames)).reset_index(drop=True)
    
#     # We will defer to table 1, so we're dropping the rows from table 2 below
#     # Now removing from table 2 the sources that also appear in table 1 by removing the indices listed in the \
#     # c2_matches table
#     t2 = (t2[~t2['index'].isin(c2_dups['idx'])]).reset_index(drop=True)
#     t2.drop(labels=['index'], axis=1, inplace=True) # Now dropping the index column as it is not needed



In [ ]:
# # Here's a function that we'll use to match catalogs together, since by this point our column names will be uniform

# def match_tables_fib(t1,t2,match_tol):
#     if 'level_0' in t1.columns:
#         t1.drop(labels=['level_0'], axis=1, inplace=True)
#     t1.reset_index(drop=False, inplace=True)
#     if 'level_0' in t2.columns:
#         t2.drop(labels=['level_0'], axis=1, inplace=True)
#     t2.reset_index(inplace=True, drop=False)
#     # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
#     c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
#     c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
#     idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
#     # idx2 are the indices in table 2 which are the closest matching rows to table 1
    
#     # Adding a match tolerance here, with user input for the function
#     max_sep = match_tol * u.arcsec # The max match tolerance will be 5'
#     sep_constraint = d2d2 <= max_sep # Filtering the angular separations from the matches to ≤ 5''
    
#     # Now applying the match tolerance to the matches for each table
#     #c1_matches = c1[sep_constraint] # applying to the matches in table 1
#     c2_matches = c2[idx2[sep_constraint]] # applying to the matches in table 2

#     #print(c1_matches)
#     #print(c2_matches)
#     # Now storing the relevant index information for later...
#     c2_matchesRA1 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in table 2
#     c2_matchesRA1['idx2'] = idx2[sep_constraint] # Storing the indices
#     c2_matchesRA1['RA1'] = c2_matches.ra.degree # Storing the coordinates in these two rows
#     c2_matchesRA1['Dec1'] = c2_matches.dec.degree
    
#     matches = pd.DataFrame()
#     matches['idx2'] = idx2[sep_constraint]
    
#     # Concatenating these frames now
#     frames = [c2_matchesRA1]
#     c2_dups = (pd.concat(frames)).reset_index(drop=True)
#     #print(t2)
#     # We will defer to table 1, so we're dropping the rows from table 2 below
#     # Now removing from table 2 the sources that also appear in table 1 by removing the indices listed in the \
#     # c2_matches table
#     t2unique = (t2[~t2['index'].isin(c2_dups['idx2'])]).reset_index(drop=True)
#     # And retaining a copy of the matches:
#     t2matches = (t2[t2['index'].isin(c2_dups['idx2'])]).reset_index(drop=True)
#     t2unique.drop(labels=['index'], axis=1, inplace=True) # Now dropping the index column as it is not needed
#     #print(c1_matches)
#     #print(c2_matches)
#     #print(t2)
    
    
#     # Need to add in some commands here that append bib and author info to table 1 before removing the rows \
#     # from table 2
#     idx1, d2d1, d3d1 = match_coordinates_sky(c2, c1) # Now matching table 1 to table 2
#     sep_constraint = d2d1 <= max_sep # Filtering the angular separations from the matches to ≤ 5''
#     c1_matches = c1[idx1[sep_constraint]] # applying to the matches in table 2
#     #print(c1_matches)
    
#     # Now storing the relevant index information for later...
#     c1_matchesRA1 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in table 2
#     c1_matchesRA1['idx1'] = idx1[sep_constraint] # Storing the indices
#     c1_matchesRA1['RA1'] = c1_matches.ra.degree # Storing the coordinates in these two rows
#     c1_matchesRA1['Dec1'] = c1_matches.dec.degree
    
#     # Concatenating these frames now
#     frames = [c1_matchesRA1]
#     c1_dups = (pd.concat(frames)).reset_index(drop=True)
    
#     matches['idx1'] = idx1[sep_constraint]
#     #matches['index'] = matches['idx1']
#     #print(matches)


#     # Now adding in info from Table 2 into Table 1
#     # First we'll remove the non-unique rows:
#     t1unique = (t1[~t1['index'].isin(c1_dups['idx1'])]).reset_index(drop=True)
#     # Next we'll make a dataframe for the matches, and begin adding in the information from table 2
#     t1matches = (t1[t1['index'].isin(c1_dups['idx1'])])#.reset_index(drop=True) #I'm not sure why I have to use . notation here instead of brackets
#     #t1
#     #print(matches)
    
#     #print(t1matches)
#     #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
#     #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
#     #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
#     t1matches.merge(matches, left_on=['index'], right_on=['idx1'])
#     #print(t1matches)
#     print(matches.dtypes)
#     #.loc[matches.loc['idx1','idx2'],
#     #.loc[matches.loc['idx1','idx2'],
#     #.loc[matches.loc['idx1','idx2'],
#     t1unique.drop(labels=['index'], axis=1, inplace=True) # Now dropping the index column as it is not needed
#     #print(c1_matchesRA1)
#     #print(c2_matches)
#     #print(matches)
#     #print(t1)

#     return t1matches, t2matches



In [ ]:
#def name_to_coords(df,dfcol): #for WANG+2009
#    # A function to take in the SDSS designation in a form JXXXXXX+XXXXXX and convert this to sexagesimal coordinates, where the '+' could be '+' or '-'.
#    # Output form will be: XX:XX:XX +XX:XX:XX, where the '+' could be '+' or '-'
#    df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
#    df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
#    df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
#    df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
#    df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
#    df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
#    return
#
## This function is for converting namings in the following format to coordinates. This is useful for later \
## catalogs 2010-2020
#
#def name_to_coords(df,dfcol): # FOR LIU+2010a
#    # A function to take in the SDSS designation in a form JXXXXXX.XX+XXXXXX.X and convert this to sexagesimal coordinates, where the '+' could be '+' or '-'.
#    # Output form will be: XX:XX:XX.XX +XX:XX:XX.X, where the '+' could be '+' or '-'
#    df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
#    df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
#    df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
#    df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#    df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#    df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
#    return
#